In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

# Replace with your own database credentials
db_config = {
    "host": "db00",
    "user": "root",
    "password": "Local1234567",
    "database": "osticket_1-10-1_live"
}
# Create a SQLAlchemy engine using the MySQL connector connection
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

# Define your SQL query
query = """
    SELECT 
        CONCAT(st.firstname, ' ', st.lastname) AS 'AssignedTo',
        te.name AS AssignedToTeam,
        t.created AS created,
        t.updated AS updated,
        Title.value AS Title,
        Classification.value AS Classification,
        Topic.value AS Topic,
        tre.body AS Text
    FROM 
        ost_ticket t
        JOIN ost_ticket_status ts ON (t.status_id = ts.id)
        JOIN ost_department d ON (t.dept_id = d.id)
        JOIN ost_thread tr ON (t.ticket_id = tr.object_id)
        JOIN ost_thread_entry tre ON (tr.id = tre.thread_id AND tre.type = 'M' AND tre.pid = 0)
        JOIN ost_form_entry f1 ON (t.ticket_id = f1.object_id AND f1.form_id = 2)
        LEFT JOIN ost_staff st ON (t.staff_id = st.staff_id)
        LEFT JOIN ost_team te ON (t.team_id = te.team_id)
        LEFT JOIN ost_form_entry_values Title ON (f1.id = Title.entry_id AND Title.field_id = 5 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Priority ON (f1.id = Priority.entry_id AND Priority.field_id = 7 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Classification ON (f1.id = Classification.entry_id AND Classification.field_id = 29 AND f1.form_id = 2)
        LEFT JOIN ost_form_entry_values Topic ON (f1.id = Topic.entry_id AND Topic.field_id = 31 AND f1.form_id = 2)
    WHERE 
        d.name = 'TABZ FLS'
    ORDER BY 
        created DESC
    LIMIT 5000
"""

# Use SQLAlchemy engine to execute the query and load data into a DataFrame
df = pd.read_sql(query, engine)


In [2]:
print(df)

              AssignedTo            AssignedToTeam             created  \
0     Francesco Zambaldi               H/S Support 2023-09-13 08:42:55   
1     Francesco Zambaldi               H/S Support 2023-09-13 08:42:23   
2        Manuel Furlotti  User Identity Management 2023-09-13 08:10:53   
3           Daniele Neri                       NAV 2023-09-13 07:58:13   
4           Daniele Neri                       NAV 2023-09-13 07:58:13   
...                  ...                       ...                 ...   
4995        Daniele Neri                       NAV 2023-03-15 16:34:56   
4996        Daniele Neri                       NAV 2023-03-15 16:20:38   
4997        Daniele Neri                       NAV 2023-03-15 16:19:25   
4998        Daniele Neri                       NAV 2023-03-15 16:19:25   
4999        Daniele Neri                       NAV 2023-03-15 16:15:21   

                 updated                                              Title  \
0    2023-09-13 08:50:06        

In [3]:
# List of columns to keep (exclude the ones you want to remove)
columns_to_keep = ['Title', 'Topic', 'Text']

# Create a new DataFrame with only the selected columns
df = df[columns_to_keep]

# df now contains only columns A and B
print(df)

                                                  Title  \
0                                 rubrica da aggiornare   
1                               Credenziali Adobe Cloud   
2                                  Badge Adriana Amodio   
3     testo sbagliato nelle VCEE di merce dove c'è d...   
4     testo sbagliato nelle VCEE di merce dove c'è d...   
...                                                 ...   
4995                              Fehler TRPT Worksheet   
4996                                      TRPT-23.00865   
4997                     Subcontracting cambiare logica   
4998                     Subcontracting cambiare logica   
4999                                      Errore Script   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"16":"User identity management, Permissions, ...   
3                                 {"22":"Dynamics NAV"}

In [4]:
import re

values_to_keep = [
    '{"16":"User identity management, Permissions, Badge, Mail"}',
    '{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}',
    '{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}'
]

# Use str.contains to create a boolean mask, replace NaN with False, and then filter
mask = df['Topic'].str.contains('|'.join(map(re.escape, values_to_keep))).fillna(False)
filtered_df = df[mask]

# Print the filtered DataFrame
print(filtered_df)

                                                 Title  \
0                                rubrica da aggiornare   
1                              Credenziali Adobe Cloud   
2                                 Badge Adriana Amodio   
5                                         New Computer   
7     prego attivare risposta automatica TA/EPT/SNOMAX   
...                                                ...   
4985                                     Email account   
4986                                     Email account   
4987                       Activate Peixi Chen's email   
4988                       Activate Peixi Chen's email   
4989                              E-Mail Signatur TAUS   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"16":"User identity management, Permissions, ...   
5     {"12":"Standard Software: Operating systems, M...   
7     {"

In [5]:
# Reset the index to start from 0
filtered_df = filtered_df.reset_index(drop=True)

# Print the filtered DataFrame
print(filtered_df)

                                                 Title  \
0                                rubrica da aggiornare   
1                              Credenziali Adobe Cloud   
2                                 Badge Adriana Amodio   
3                                         New Computer   
4     prego attivare risposta automatica TA/EPT/SNOMAX   
...                                                ...   
2735                                     Email account   
2736                                     Email account   
2737                       Activate Peixi Chen's email   
2738                       Activate Peixi Chen's email   
2739                              E-Mail Signatur TAUS   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"16":"User identity management, Permissions, ...   
3     {"12":"Standard Software: Operating systems, M...   
4     {"

In [6]:
from IPython.display import display, HTML

display(HTML('<style>.output { overflow-x: scroll; white-space: nowrap; }</style>'))
filtered_df

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...","<p>Ciao,</p> <p>chiedo cortesemente venga aggi..."
1,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...","<p>Buongiorno,</p> <p>si richiedono credenzial..."
2,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...","<p>Buongiorno,</p> <p>vi prego di dare al Badg..."
3,New Computer,"{""12"":""Standard Software: Operating systems, M...",<p>I have a new laptop that I need added to th...
4,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",<p>Prego attivare la risposta automatica a par...
...,...,...,...
2735,Email account,"{""16"":""User identity management, Permissions, ...","<p>Dear colleague,</p> <p>Please help to creat..."
2736,Email account,"{""16"":""User identity management, Permissions, ...",<p><span><span>Hi Manuel</span></span></p> <p>...
2737,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...","<p>Dear Colleague,</p> <p>Please activate peix..."
2738,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",<p><span><span><span><span>嗨丹妮尔，</span></span>...


In [7]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
#keep only the text

In [9]:
# Assuming you have a DataFrame named 'df' and you want to see all rows of the 'column_name' column
column_name = 'Text'
pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_df[column_name])
pd.reset_option('display.max_rows')  # Reset the option to its default value

0       <p>Ciao,</p> <p>chiedo cortesemente venga aggi...
1       <p>Buongiorno,</p> <p>si richiedono credenzial...
2       <p>Buongiorno,</p> <p>vi prego di dare al Badg...
3       <p>I have a new laptop that I need added to th...
4       <p>Prego attivare la risposta automatica a par...
5       <p>hello</p> <p>did you from the edv departmen...
6       <p>hello</p> <p>I have removed a USB stick tha...
7       <figure><img src="cid:xwual1utjl_i575j8wotsa1h...
8       <figure><img src="cid:xesmpdbl2jgd9lfeff_rs3yg...
9       <figure><img src="cid:spcfzf-cq4mfn4o0vf_gjnur...
10      <p>Guten Morgen</p> <p>Der Neustart hat gester...
11      <p>Gestern Mittag 12 Uhr habe ich meinen Rechn...
12      <p>The stick contains word, excel and pdf - da...
13      <p>"Honestly i can't recover files from client...
14      <p>Durante il viaggio in finlandia 2 settimana...
15      <p>Procall mi da il seguente errore... anche f...
16      <p>"DHCP Server not found" e il errore sul dis...
17       <div 

In [10]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [11]:
def remove_html_tags(text):
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    
    # Remove HTML tags and extract only the text
    text = soup.get_text()
    
    # Remove any remaining special characters
    pattern = r'[^a-zA-Z0-9\s]'
    return re.sub(pattern, '', text)

# Apply the remove_html_tags function to the DataFrame columns containing HTML data
filtered_df['Text'] = filtered_df['Text'].apply(remove_html_tags)


C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\1070994409.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [12]:
column_name = 'Text'
pd.set_option('display.max_rows', None)  # Set the option to display all rows
print(filtered_df[column_name])
pd.reset_option('display.max_rows')  # Reset the option to its default value

0       Ciao chiedo cortesemente venga aggiornata la r...
1       Buongiorno si richiedono credenziali per Adobe...
2       Buongiorno vi prego di dare al Badge di Adrian...
3       I have a new laptop that I need added to the d...
4       Prego attivare la risposta automatica a partir...
5       hello did you from the edv department push som...
6       hello I have removed a USB stick that I use ex...
7                                                        
8                                                        
9                                                        
10      Guten Morgen Der Neustart hat gestern lange ge...
11      Gestern Mittag 12 Uhr habe ich meinen Rechner ...
12      The stick contains word excel and pdf  data fr...
13      Honestly i cant recover files from clients HDD...
14      Durante il viaggio in finlandia 2 settimana fa...
15      Procall mi da il seguente errore anche facendo...
16      DHCP Server not found e il errore sul display ...
17         Hal

In [13]:
filtered_df.to_csv('cleaned_data.csv', index=False)

In [14]:
df = pd.read_csv('cleaned_data.csv')
df

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao chiedo cortesemente venga aggiornata la r...
1,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",Buongiorno si richiedono credenziali per Adobe...
2,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",Buongiorno vi prego di dare al Badge di Adrian...
3,New Computer,"{""12"":""Standard Software: Operating systems, M...",I have a new laptop that I need added to the d...
4,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",Prego attivare la risposta automatica a partir...
...,...,...,...
2735,Email account,"{""16"":""User identity management, Permissions, ...",Dear colleague Please help to creat an email a...
2736,Email account,"{""16"":""User identity management, Permissions, ...",Hi Manuel Cau you please send me the password ...
2737,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Dear Colleague Please activate peixichentechno...
2738,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Yes he needs both now


In [15]:
df = df.dropna()

# The df_cleaned DataFrame will contain only rows with no NaN values
df

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao chiedo cortesemente venga aggiornata la r...
1,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",Buongiorno si richiedono credenziali per Adobe...
2,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",Buongiorno vi prego di dare al Badge di Adrian...
3,New Computer,"{""12"":""Standard Software: Operating systems, M...",I have a new laptop that I need added to the d...
4,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",Prego attivare la risposta automatica a partir...
...,...,...,...
2735,Email account,"{""16"":""User identity management, Permissions, ...",Dear colleague Please help to creat an email a...
2736,Email account,"{""16"":""User identity management, Permissions, ...",Hi Manuel Cau you please send me the password ...
2737,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Dear Colleague Please activate peixichentechno...
2738,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Yes he needs both now


In [16]:
df.to_csv('final_data.csv', index=False)

In [17]:
df = pd.read_csv('final_data.csv')
df

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao chiedo cortesemente venga aggiornata la r...
1,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",Buongiorno si richiedono credenziali per Adobe...
2,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",Buongiorno vi prego di dare al Badge di Adrian...
3,New Computer,"{""12"":""Standard Software: Operating systems, M...",I have a new laptop that I need added to the d...
4,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",Prego attivare la risposta automatica a partir...
...,...,...,...
2600,Email account,"{""16"":""User identity management, Permissions, ...",Dear colleague Please help to creat an email a...
2601,Email account,"{""16"":""User identity management, Permissions, ...",Hi Manuel Cau you please send me the password ...
2602,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Dear Colleague Please activate peixichentechno...
2603,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Yes he needs both now


In [18]:
#DELETE CONFIDENTIAL MESSAGE

import pandas as pd
import re

# Create or load your DataFrame 'df' with columns 'Title', 'Topic', and 'Text'

# Define a regular expression pattern to match the confidential message
confidential_pattern = re.compile(r"(?i)THIS MESSAGE INCLUDING ANY ATTACHMENTS CONTAINS CONFIDENTIAL INFORMATION.*?PROHIBITED", re.DOTALL)

# Function to remove text containing the confidential message
def remove_confidential_text(text):
    return re.sub(confidential_pattern, '', text)

# Apply the function to the 'Text' column
df['Text'] = df['Text'].apply(remove_confidential_text)

# Display the modified DataFrame
print(df)


                                                 Title  \
0                                rubrica da aggiornare   
1                              Credenziali Adobe Cloud   
2                                 Badge Adriana Amodio   
3                                         New Computer   
4     prego attivare risposta automatica TA/EPT/SNOMAX   
...                                                ...   
2600                                     Email account   
2601                                     Email account   
2602                       Activate Peixi Chen's email   
2603                       Activate Peixi Chen's email   
2604                              E-Mail Signatur TAUS   

                                                  Topic  \
0     {"11":"Hardware, Telephony, Smartphones, Monit...   
1     {"12":"Standard Software: Operating systems, M...   
2     {"16":"User identity management, Permissions, ...   
3     {"12":"Standard Software: Operating systems, M...   
4     {"

In [19]:
import pandas as pd

desired_value = r'{"16":"User identity management, Permissions, Badge, Mail"}'

df_1 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_1

,Title,Topic,Text
2,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",Buongiorno vi prego di dare al Badge di Adrian...
4,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",Prego attivare la risposta automatica a partir...
17,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopossiamo per favore togliere la licenza CR...
18,New AD-Group per DMS,"{""16"":""User identity management, Permissions, ...",Ciaomi potete per favore creare un nuovo grupp...
20,Email Signature - Immagine,"{""16"":""User identity management, Permissions, ...",Salve sarebbe da aggiungere su tutti gli utent...
...,...,...,...
2600,Email account,"{""16"":""User identity management, Permissions, ...",Dear colleague Please help to creat an email a...
2601,Email account,"{""16"":""User identity management, Permissions, ...",Hi Manuel Cau you please send me the password ...
2602,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Dear Colleague Please activate peixichentechno...
2603,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Yes he needs both now


In [20]:
# Reset the index to start from 0
df_1 = df_1.reset_index(drop=True)
df_1

,Title,Topic,Text
0,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",Buongiorno vi prego di dare al Badge di Adrian...
1,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",Prego attivare la risposta automatica a partir...
2,Licenze CRM,"{""16"":""User identity management, Permissions, ...",Ciaopossiamo per favore togliere la licenza CR...
3,New AD-Group per DMS,"{""16"":""User identity management, Permissions, ...",Ciaomi potete per favore creare un nuovo grupp...
4,Email Signature - Immagine,"{""16"":""User identity management, Permissions, ...",Salve sarebbe da aggiungere su tutti gli utent...
...,...,...,...
902,Email account,"{""16"":""User identity management, Permissions, ...",Dear colleague Please help to creat an email a...
903,Email account,"{""16"":""User identity management, Permissions, ...",Hi Manuel Cau you please send me the password ...
904,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Dear Colleague Please activate peixichentechno...
905,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",Yes he needs both now


In [21]:
import pandas as pd

# Convert the 'Text' column to uppercase
df_1['Text'] = df_1['Text'].str.upper()

df_1

,Title,Topic,Text
0,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIAN...
1,prego attivare risposta automatica TA/EPT/SNOMAX,"{""16"":""User identity management, Permissions, ...",PREGO ATTIVARE LA RISPOSTA AUTOMATICA A PARTIR...
2,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOSSIAMO PER FAVORE TOGLIERE LA LICENZA CR...
3,New AD-Group per DMS,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE PER FAVORE CREARE UN NUOVO GRUPP...
4,Email Signature - Immagine,"{""16"":""User identity management, Permissions, ...",SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENT...
...,...,...,...
902,Email account,"{""16"":""User identity management, Permissions, ...",DEAR COLLEAGUE PLEASE HELP TO CREAT AN EMAIL A...
903,Email account,"{""16"":""User identity management, Permissions, ...",HI MANUEL CAU YOU PLEASE SEND ME THE PASSWORD ...
904,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",DEAR COLLEAGUE PLEASE ACTIVATE PEIXICHENTECHNO...
905,Activate Peixi Chen's email,"{""16"":""User identity management, Permissions, ...",YES HE NEEDS BOTH NOW


In [22]:
import pandas as pd

desired_value = r'{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}'  # Note the 'r' before the string for a raw string

df_2 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_2

,Title,Topic,Text
1,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",Buongiorno si richiedono credenziali per Adobe...
3,New Computer,"{""12"":""Standard Software: Operating systems, M...",I have a new laptop that I need added to the d...
5,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",hello did you from the edv department push som...
6,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",hello I have removed a USB stick that I use ex...
7,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",Guten Morgen Der Neustart hat gestern lange ge...
...,...,...,...
2570,P&ID database error,"{""12"":""Standard Software: Operating systems, M...",Hello I have problem when clicking to symbols ...
2571,Navision,"{""12"":""Standard Software: Operating systems, M...",aprendo un TRPT il sistema mi da sempre questo...
2572,SAP Concur,"{""12"":""Standard Software: Operating systems, M...",Ciao messaggio invalid csrf token intercepted ...
2595,Microsoft Outlook - Mancano le firme,"{""12"":""Standard Software: Operating systems, M...",Ciao su Outlook del mio nuovo laptop L779 manc...


In [23]:
# Reset the index to start from 0
df_2 = df_2.reset_index(drop=True)
df_2

,Title,Topic,Text
0,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",Buongiorno si richiedono credenziali per Adobe...
1,New Computer,"{""12"":""Standard Software: Operating systems, M...",I have a new laptop that I need added to the d...
2,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",hello did you from the edv department push som...
3,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",hello I have removed a USB stick that I use ex...
4,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",Guten Morgen Der Neustart hat gestern lange ge...
...,...,...,...
809,P&ID database error,"{""12"":""Standard Software: Operating systems, M...",Hello I have problem when clicking to symbols ...
810,Navision,"{""12"":""Standard Software: Operating systems, M...",aprendo un TRPT il sistema mi da sempre questo...
811,SAP Concur,"{""12"":""Standard Software: Operating systems, M...",Ciao messaggio invalid csrf token intercepted ...
812,Microsoft Outlook - Mancano le firme,"{""12"":""Standard Software: Operating systems, M...",Ciao su Outlook del mio nuovo laptop L779 manc...


In [24]:
df_2['Text'] = df_2['Text'].str.upper()

df_2

,Title,Topic,Text
0,Credenziali Adobe Cloud,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO SI RICHIEDONO CREDENZIALI PER ADOBE...
1,New Computer,"{""12"":""Standard Software: Operating systems, M...",I HAVE A NEW LAPTOP THAT I NEED ADDED TO THE D...
2,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",HELLO DID YOU FROM THE EDV DEPARTMENT PUSH SOM...
3,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",HELLO I HAVE REMOVED A USB STICK THAT I USE EX...
4,nach Update fehlten die Unterordner des Arbeit...,"{""12"":""Standard Software: Operating systems, M...",GUTEN MORGEN DER NEUSTART HAT GESTERN LANGE GE...
...,...,...,...
809,P&ID database error,"{""12"":""Standard Software: Operating systems, M...",HELLO I HAVE PROBLEM WHEN CLICKING TO SYMBOLS ...
810,Navision,"{""12"":""Standard Software: Operating systems, M...",APRENDO UN TRPT IL SISTEMA MI DA SEMPRE QUESTO...
811,SAP Concur,"{""12"":""Standard Software: Operating systems, M...",CIAO MESSAGGIO INVALID CSRF TOKEN INTERCEPTED ...
812,Microsoft Outlook - Mancano le firme,"{""12"":""Standard Software: Operating systems, M...",CIAO SU OUTLOOK DEL MIO NUOVO LAPTOP L779 MANC...


In [25]:
import pandas as pd

desired_value = r'{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}'

df_3 = df[df['Topic'].str.contains(desired_value, regex=True)]
df_3

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao chiedo cortesemente venga aggiornata la r...
11,Problem with the connection HOTSPOT to my comp...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Durante il viaggio in finlandia 2 settimana fa...
13,Panasonic Telephone has no connection,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DHCP Server not found e il errore sul display ...
15,Cellulare senza rete,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao il mio Samsung non prende pi rete interne...
16,Monitor,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi we found this type of monitor for a good pr...
...,...,...,...
2586,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hello my printers are offline again Im in the ...
2587,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",When I shut down my laptop the printer starts ...
2596,Drucker brauch über 20 Sekunden,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Nachdem man Drucken drckt braucht der Drucker ...
2598,Plotter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",non riesco piu a stampare direttamente sul plo...


In [26]:
# Reset the index to start from 0
df_3 = df_3.reset_index(drop=True)
df_3

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao chiedo cortesemente venga aggiornata la r...
1,Problem with the connection HOTSPOT to my comp...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Durante il viaggio in finlandia 2 settimana fa...
2,Panasonic Telephone has no connection,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DHCP Server not found e il errore sul display ...
3,Cellulare senza rete,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Ciao il mio Samsung non prende pi rete interne...
4,Monitor,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hi we found this type of monitor for a good pr...
...,...,...,...
879,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Hello my printers are offline again Im in the ...
880,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",When I shut down my laptop the printer starts ...
881,Drucker brauch über 20 Sekunden,"{""11"":""Hardware, Telephony, Smartphones, Monit...",Nachdem man Drucken drckt braucht der Drucker ...
882,Plotter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",non riesco piu a stampare direttamente sul plo...


In [27]:
df_3['Text'] = df_3['Text'].str.upper()

df_3

,Title,Topic,Text
0,rubrica da aggiornare,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CHIEDO CORTESEMENTE VENGA AGGIORNATA LA R...
1,Problem with the connection HOTSPOT to my comp...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DURANTE IL VIAGGIO IN FINLANDIA 2 SETTIMANA FA...
2,Panasonic Telephone has no connection,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DHCP SERVER NOT FOUND E IL ERRORE SUL DISPLAY ...
3,Cellulare senza rete,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO IL MIO SAMSUNG NON PRENDE PI RETE INTERNE...
4,Monitor,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI WE FOUND THIS TYPE OF MONITOR FOR A GOOD PR...
...,...,...,...
879,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO MY PRINTERS ARE OFFLINE AGAIN IM IN THE ...
880,Plotter wird nicht erkannt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WHEN I SHUT DOWN MY LAPTOP THE PRINTER STARTS ...
881,Drucker brauch über 20 Sekunden,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NACHDEM MAN DRUCKEN DRCKT BRAUCHT DER DRUCKER ...
882,Plotter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NON RIESCO PIU A STAMPARE DIRETTAMENTE SUL PLO...


# MOST USED WORDS

# df_1
{"16":"User identity management, Permissions, Badge, Mail"}

In [28]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

In [29]:
import nltk
nltk.download('stopwords')

# Tokenize and remove stopwords
stop_words = set(stopwords.words('english'))
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize and remove stopwords for the 'Text' column
stop_words = set(stopwords.words('english'))
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in stop_words])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_1['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(100)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('per', 324), ('di', 312), ('grazie', 272), ('e', 245), ('il', 236), ('ciao', 234), ('non', 212), ('la', 187), ('che', 182), ('badge', 141), ('da', 123), ('mi', 120), ('bitte', 117), ('hallo', 108), ('mail', 108), ('come', 103), ('die', 102), ('le', 98), ('un', 98), ('prego', 96), ('al', 96), ('saluti', 94), ('technoalpin', 93), ('email', 92), ('danke', 84), ('favore', 83), ('please', 82), ('con', 80), ('von', 80), ('una', 78), ('39', 76), ('und', 74), ('via', 73), ('ich', 72), ('fr', 71), ('ha', 68), ('potete', 67), ('mille', 66), ('anche', 65), ('der', 65), ('nicht', 63), ('password', 62), ('2', 61), ('1', 60), ('hello', 58), ('outlook', 56), ('se', 55), ('ho', 53), ('0471', 53), ('accesso', 52), ('manuel', 52), ('del', 51), ('hi', 51), ('sono', 50), ('licenza', 49), ('den', 49), ('vi', 48), ('si', 46), ('ta', 44), ('ad', 44), ('solo', 44), ('das', 44), ('id', 44), ('agostini', 43), ('im', 43), ('nuovo', 41), ('buongiorno', 39), ('account', 39), ('0', 38), ('horst', 37), ('alle', 36

In [31]:
#It includes a custom list of words to remove

import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella']


# Tokenize and remove stopwords from the 'Text' column
df_1['tokens'] = df_1['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_1['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('badge', 141), ('mail', 108), ('email', 92), ('password', 62), ('outlook', 56), ('accesso', 52), ('licenza', 49), ('account', 39), ('pc', 33), ('accedere', 32), ('access', 30), ('diritti', 30), ('nav', 27), ('computer', 25), ('vpn', 25), ('crm', 24), ('server', 24), ('entrata', 22), ('sap', 21), ('mensa', 21), ('porte', 20), ('send', 20), ('concur', 20), ('message', 20), ('microsoft', 19), ('laptop', 19), ('phone', 18), ('allegato', 17), ('wie', 17), ('3', 16)]


In [32]:
import pandas as pd

# Create keywords for each subset
keywords = {
    'df_1_email': ['message', 'mail', 'email', 'emails', 'account', 'sender', 'recipient',
                   'microsoft', 'outlook', 'password', 'attachments', 'send','licenza'],
    'df_2_rights': ['badge', 'accedere', 'diritti', 'access', 'porte', 'mensa', 'accesso'],
    'df_3_computer': ['pc', 'laptop', 'computer', 'phone', 'password'],
    'df_4_software': ['server', 'nav', 'vpn', 'crm', 'sap', 'concur']
}

# Initialize empty DataFrames for each subset
subset_dfs = {key: pd.DataFrame(columns=['Text']) for key in keywords.keys()}

# Iterate through each row in df_1 and assign it to the appropriate subset
for index, row in df_1.iterrows():
    text = row['Text']
    for subset_name, subset_keywords in keywords.items():
        if any(keyword in text.lower() for keyword in subset_keywords):
            subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)

# Now, you have four subsets in separate DataFrames
df_1_email = subset_dfs['df_1_email']
df_2_rights = subset_dfs['df_2_rights']
df_3_computer = subset_dfs['df_3_computer']
df_4_software = subset_dfs['df_4_software']

# You can access and work with each subset as needed
print("df_1_email:")
print(df_1_email)

print("\ndf_2_rights:")
print(df_2_rights)

print("\ndf_3_computer:")
print(df_3_computer)

print("\ndf_4_software:")
print(df_4_software)


C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\4151013874.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subset_dfs[subset_name] = subset_dfs[subset_name].append({'Text': text}, ignore_index=True)
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\4151013874.py:20: FutureWarning: The frame.append method 

df_1_email:
                                                  Text
0    PREGO ATTIVARE LA RISPOSTA AUTOMATICA A PARTIR...
1    CIAOPOSSIAMO PER FAVORE TOGLIERE LA LICENZA CR...
2    SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENT...
3    HALLOBITTE FR UNSEREN NEUEN MITARBEITER TOBIAS...
4    KANNST DU BITTE NOCH MICROSOFT OFFICE UND TEAM...
..                                                 ...
312  HALLO MEHRERE MITARBEITER HABEN KEINEN ZUGANG ...
313  CIAO ANCORA NON VEDO LA PEC SUL MIO OUTLOOK VO...
314  CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...
315  DEAR COLLEAGUE PLEASE HELP TO CREAT AN EMAIL A...
316  HI MANUEL CAU YOU PLEASE SEND ME THE PASSWORD ...

[317 rows x 1 columns]

df_2_rights:
                                                  Text
0    BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIAN...
1    CIAO PREGO DARE ALEXA COLLAVO ACCESSO ALLA GAR...
2    CIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA...
3    HALLOBITTE FR UNSEREN NEUEN MITARBEITER TOBIAS...
4    HI I CANNO

## df_1_email

In [33]:
import pandas as pd

words_to_check = ['message', 'mail', 'email', 'emails', 'account', 'sender', 'recipient',
                   'microsoft', 'outlook', 'password', 'attachments', 'send','licenza']
# Initialize an empty DataFrame to store the matching rows
df_1_email = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_email = df_1_email.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_email = df_1_email.reset_index(drop=True)

df_1_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3047725368.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_email = df_1_email.append(df_1[

,Title,Topic,Text,tokens
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...,"[receiving, error, message, surface, opening, ..."
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...,"[karima, guerbaa, absent, until, september, 15..."
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...,"[would, like, retrieve, outlook, 2022, messages]"
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
...,...,...,...,...
547,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE LA LICENZA DI ...,"[ciaopotete, licenza, qiong, jia, francesca, t..."
548,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...,"[ciaopotremmo, licenza, bill, wang, jason, wu,..."
549,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...,"[ciaopotete, schgaguler, licenza, member, crmi..."
550,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...,"[ciaopotete, licenza, full, crm, florian, mayr..."


In [34]:
df_1_email

,Title,Topic,Text,tokens
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...,"[receiving, error, message, surface, opening, ..."
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...,"[karima, guerbaa, absent, until, september, 15..."
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...,"[would, like, retrieve, outlook, 2022, messages]"
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...,"[very, sorry, outlook, still, closing, its, ow..."
...,...,...,...,...
547,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE LA LICENZA DI ...,"[ciaopotete, licenza, qiong, jia, francesca, t..."
548,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...,"[ciaopotremmo, licenza, bill, wang, jason, wu,..."
549,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...,"[ciaopotete, schgaguler, licenza, member, crmi..."
550,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...,"[ciaopotete, licenza, full, crm, florian, mayr..."


In [35]:
df_1_email = df_1_email.drop(columns='tokens')
df_1_email

,Title,Topic,Text
0,Receiving error message on Surface,"{""16"":""User identity management, Permissions, ...",I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHE...
1,Outlook,"{""16"":""User identity management, Permissions, ...",HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15...
2,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...
3,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...
4,outlook is continuously closing automatically,"{""16"":""User identity management, Permissions, ...",HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSIN...
...,...,...,...
547,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE LA LICENZA DI ...
548,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...
549,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...
550,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE UNA LICENZA FU...


# Further refinement

## look for df_1_email    'access'

In [36]:
import pandas as pd
import re


In [37]:
related_words = ['access', 'accessing', 'accessible', 'accessed', 'accesses', 'entry', 
                 'approach', 'admission', 'entrance', 'reach', 'open', 'connect', 
                 'visit', 'retrieve', 'use','Zugang', 'Zugriff', 'zugänglich',
                 'zugriffen', 'Zugänge', 'Eingang', 'Herangehensweise', 'Zulassung',
                 'Eingang', 'erreichen', 'öffnen', 'verbinden', 'besuchen', 'abrufen', 'benutzen',
                 'accesso', 'accesso', 'accessibile', 'accesso', 'accessi', 
                 'entrata', 'approccio', 'ammissione', 'ingresso', 'raggiungere', 
                 'aprire', 'collegare', 'visitare', 'recuperare', 'usare']

In [38]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_1_email = df_1_email[df_1_email['Text'].str.contains(pattern, case=False)]
filtered_df_1_email = filtered_df_1_email.reset_index(drop=True)

filtered_df_1_email

,Title,Topic,Text
0,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...
1,Nuovo utente shared con maibox per Power BI - ...,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL AVREI BISOGNO DI UN NUOVO UTENTE ...
2,amr@technoalpin.com,"{""16"":""User identity management, Permissions, ...",CIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL AMR...
3,nuovo utente BI per Service Nordic,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL MI POTRESTI CREARE UN UTENTE CON P...
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...
...,...,...,...
81,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOSSIAMO PER FAVORE ASSEGNARE A MATTIA RIC...
82,New Entry - Elias Malfertheiner,"{""16"":""User identity management, Permissions, ...",CIAO LUNED PROSSIMA SETTIMANA INIZIA UN NUOVO ...
83,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOSSIAMO PER FAVORE ASSEGNARE UNA LICENZA ...
84,Disegnatore esterno,"{""16"":""User identity management, Permissions, ...",COME PU UNA PERSONA ESTERNA USARE LA NOSTRA LI...


In [39]:
filtered_df_1_email

,Title,Topic,Text
0,Mail history,"{""16"":""User identity management, Permissions, ...",HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022...
1,Nuovo utente shared con maibox per Power BI - ...,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL AVREI BISOGNO DI UN NUOVO UTENTE ...
2,amr@technoalpin.com,"{""16"":""User identity management, Permissions, ...",CIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL AMR...
3,nuovo utente BI per Service Nordic,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL MI POTRESTI CREARE UN UTENTE CON P...
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...
...,...,...,...
81,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOSSIAMO PER FAVORE ASSEGNARE A MATTIA RIC...
82,New Entry - Elias Malfertheiner,"{""16"":""User identity management, Permissions, ...",CIAO LUNED PROSSIMA SETTIMANA INIZIA UN NUOVO ...
83,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOSSIAMO PER FAVORE ASSEGNARE UNA LICENZA ...
84,Disegnatore esterno,"{""16"":""User identity management, Permissions, ...",COME PU UNA PERSONA ESTERNA USARE LA NOSTRA LI...


In [40]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_1_email[i:i+n] for i in range(0,len(filtered_df_1_email),n)]
    #print(list_df)

In [41]:
len(list_df)


18

In [42]:
list_df[17]


,Title,Topic,Text
85,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE ASSEGNARE A BLANDINE M...


In [43]:
def put_together():
    list1=[]
    for index in range(0,17):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [44]:
summ_block = put_together()
summ_block

Result 0:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUCIAO MANUEL  AVREI BISOGNO DI UN NUOVO UTENTE AD PER I PROJECT MANAGER CHE POSSONO USARE IN MODALIT CONDIVISA PER POWER BI EMAIL PROJECTMANAGERTECHNOALPINCOM UTENTE ALPINBIPM PASSWORT RANDOMIZZATA  GRAZIE RENCIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL AMRTECHNOALPINCOM TANTI SALUTICIAO MANUEL MI POTRESTI CREARE UN UTENTE CON PASSWORD E EMAIL IN ACTIVE DIRECTORY CHE POTREI POI USARE COME UTENTE BI PER IL SERVICE IN TA NORDIC DIREI CHE USARE LA SEGUENTE STRUTTURE PER LA MAIL E LUTENTE MAIL SERVICENOTECHNOALPINCOM ADUSER ALPINBISERVICENO  GRAZIE RENHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVAN

Result 1:
CIAOMI POTETE PREGO DARE ACCESSO SULLOUTLOOK DI ALESSANDRA PINGI LEMAIL INFOTECHNOALPINCOM CHE LEI AIUTO OGNI GG IN FOYER DA LUNEDI IN POI  E INOLTRE PREGO INCLUDERE IL NUMERO FI

['HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUCIAO MANUEL  AVREI BISOGNO DI UN NUOVO UTENTE AD PER I PROJECT MANAGER CHE POSSONO USARE IN MODALIT CONDIVISA PER POWER BI EMAIL PROJECTMANAGERTECHNOALPINCOM UTENTE ALPINBIPM PASSWORT RANDOMIZZATA  GRAZIE RENCIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL\xa0AMRTECHNOALPINCOM TANTI SALUTICIAO MANUEL MI POTRESTI CREARE UN UTENTE CON PASSWORD E\xa0EMAIL IN ACTIVE DIRECTORY CHE POTREI POI USARE COME UTENTE BI PER IL SERVICE IN TA NORDIC DIREI CHE USARE LA SEGUENTE STRUTTURE PER LA MAIL E LUTENTE MAIL SERVICENOTECHNOALPINCOM ADUSER ALPINBISERVICENO  GRAZIE RENHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVAN',
 'CIAOMI POTETE PREGO DARE ACCESSO SULLOUTLOOK DI ALESSANDRA PINGI LEMAIL INFOTECHNOALPINCOM CHE LEI AIUTO OGNI GG IN FOYER DA LUNEDI IN POI\xa0 E INOLTRE PREGO INCLUDERE IL NUMERO FISSO DI

In [45]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 17):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [46]:
blocks_to_skip = [15]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)


Result 0:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUCIAO MANUEL  AVREI BISOGNO DI UN NUOVO UTENTE AD PER I PROJECT MANAGER CHE POSSONO USARE IN MODALIT CONDIVISA PER POWER BI EMAIL PROJECTMANAGERTECHNOALPINCOM UTENTE ALPINBIPM PASSWORT RANDOMIZZATA  GRAZIE RENCIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL AMRTECHNOALPINCOM TANTI SALUTICIAO MANUEL MI POTRESTI CREARE UN UTENTE CON PASSWORD E EMAIL IN ACTIVE DIRECTORY CHE POTREI POI USARE COME UTENTE BI PER IL SERVICE IN TA NORDIC DIREI CHE USARE LA SEGUENTE STRUTTURE PER LA MAIL E LUTENTE MAIL SERVICENOTECHNOALPINCOM ADUSER ALPINBISERVICENO  GRAZIE RENHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVAN

Result 1:
CIAOMI POTETE PREGO DARE ACCESSO SULLOUTLOOK DI ALESSANDRA PINGI LEMAIL INFOTECHNOALPINCOM CHE LEI AIUTO OGNI GG IN FOYER DA LUNEDI IN POI  E INOLTRE PREGO INCLUDERE IL NUMERO FI

In [47]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUCIAO MANUEL  AVREI BISOGNO DI UN NUOVO UTENTE AD PER I PROJECT MANAGER CHE POSSONO USARE IN MODALIT CONDIVISA PER POWER BI EMAIL PROJECTMANAGERTECHNOALPINCOM UTENTE ALPINBIPM PASSWORT RANDOMIZZATA  GRAZIE RENCIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL AMRTECHNOALPINCOM TANTI SALUTICIAO MANUEL MI POTRESTI CREARE UN UTENTE CON PASSWORD E EMAIL IN ACTIVE DIRECTORY CHE POTREI POI USARE COME UTENTE BI PER IL SERVICE IN TA NORDIC DIREI CHE USARE LA SEGUENTE STRUTTURE PER LA MAIL E LUTENTE MAIL SERVICENOTECHNOALPINCOM ADUSER ALPINBISERVICENO  GRAZIE RENHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVAN

Result 1:
CIAOMI POTETE PREGO DARE ACCESSO SULLOUTLOOK DI ALESSANDRA PINGI LEMAIL INFOTECHNOALPINCOM CHE LEI AIUTO OGNI GG IN FOYER DA LUNEDI IN POI  E INOLTRE PREGO INCLUDERE IL NUMERO FI

['HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUCIAO MANUEL  AVREI BISOGNO DI UN NUOVO UTENTE AD PER I PROJECT MANAGER CHE POSSONO USARE IN MODALIT CONDIVISA PER POWER BI EMAIL PROJECTMANAGERTECHNOALPINCOM UTENTE ALPINBIPM PASSWORT RANDOMIZZATA  GRAZIE RENCIAO PREGO DARMI ACCESSO ALLINDIRIZZO MAIL\xa0AMRTECHNOALPINCOM TANTI SALUTICIAO MANUEL MI POTRESTI CREARE UN UTENTE CON PASSWORD E\xa0EMAIL IN ACTIVE DIRECTORY CHE POTREI POI USARE COME UTENTE BI PER IL SERVICE IN TA NORDIC DIREI CHE USARE LA SEGUENTE STRUTTURE PER LA MAIL E LUTENTE MAIL SERVICENOTECHNOALPINCOM ADUSER ALPINBISERVICENO  GRAZIE RENHELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVAN',
 'CIAOMI POTETE PREGO DARE ACCESSO SULLOUTLOOK DI ALESSANDRA PINGI LEMAIL INFOTECHNOALPINCOM CHE LEI AIUTO OGNI GG IN FOYER DA LUNEDI IN POI\xa0 E INOLTRE PREGO INCLUDERE IL NUMERO FISSO DI

In [48]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


# df_1_email_ signature

In [159]:
import pandas as pd
import re

In [160]:
related_words = [
    'Firma', 'Autografo', 'Sottoscrizione', 'Segno distintivo', 'Sigla', 'Marcatura',
    'Ratifica', 'Apposizione di firma', 'Sigillo', 'Atto autografo',
    'Signature', 'Autograph', 'Mark', 'Seal', 'Sign-off',
    'Inked mark', 'Signatory', 'Handwriting', 'Personal mark',
    'Unterschrift', 'Autogramm', 'Kennzeichen', 'Siegel', 'Namenszug', 'Paraphe',
    'Besiegelung', 'Unterzeichnung', 'Handschrift', 'Eigenhändige Unterschrift'
]


In [161]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_1_email_signature = df_1_email[df_1_email['Text'].str.contains(pattern, case=False)]
filtered_df_1_email_signature = filtered_df_1_email_signature.reset_index(drop=True)

filtered_df_1_email_signature

,Title,Topic,Text
0,Email Signature - Immagine,"{""16"":""User identity management, Permissions, ...",SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENT...
1,Mail without signature,"{""16"":""User identity management, Permissions, ...",I CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAIL
2,Mail signature,"{""16"":""User identity management, Permissions, ...",HALLO COULD YOU CREATE ME A OUTLOOK EMAIL SI...
3,Mail Signatur,"{""16"":""User identity management, Permissions, ...",PREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL ...
4,Richiesta accessi,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL ...
5,Problema arrivo ed invio mail,"{""16"":""User identity management, Permissions, ...",CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SE...
6,Firma Email,"{""16"":""User identity management, Permissions, ...",CIAO MI SONO ACCORTO CHE NON VIENE INSERITA LA...
7,New Entry - Elias Malfertheiner,"{""16"":""User identity management, Permissions, ...",CIAO LUNED PROSSIMA SETTIMANA INIZIA UN NUOVO ...
8,Email Signature,"{""16"":""User identity management, Permissions, ...",CIAO MI POTETE ANCORA AGGIUNGERE LEMAIL SIGNAT...
9,firma email,"{""16"":""User identity management, Permissions, ...",CIAO MI POTETE SISTEMARE PER FAVORE LA FIRMA N...


In [164]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_1_email_signature[i:i+n] for i in range(0,len(filtered_df_1_email_signature),n)]
    #print(list_df)


In [165]:
len(list_df)

8

In [167]:
list_df[7]

,Title,Topic,Text
35,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO MANUEL ANTOINE HA COMINCIATO OGGI POTREST...
36,New Entry - Elias Malfertheiner,"{""16"":""User identity management, Permissions, ...",CIAO LUNED PROSSIMA SETTIMANA INIZIA UN NUOVO ...


In [168]:
def put_together():
    list1=[]
    for index in range(0,7):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENTI ENGO QUESTA IMMAGINE NELLA EMAIL SIGNATURE  GRAZIE E SALUTI TOBIAI CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAILHALLO   COULD YOU CREATE ME A OUTLOOK EMAIL SIGNATURE PLEASE THANK YOU TOMASPREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WARENEINGANG SIEMENSSTRAE 11082023  18082023 WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023 CHIUSO PER FERIEGOODS RECEIPT VIA SIEMENS 11082023  18082023 CLOSED FOR HOLIDAY  PER MARTINP ARTHURS PATRICKT VANESSAM SILVIAZ E SAMUELN  E SOLO DA 020823 AL 110823   BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL MIO ACCOUNT I SEGUENTI ACCESSI  INSERIRE DISCO G NEL MIO ACCOUNT  AGGIUNGERE LA FIRMA ALLA MAIL  AGGIUNGERE IL PROGRAMMA PROCALL AL MIO ACCOUNT GRAZIE E BUON LAVORO LAURA SPAZIAN

Result 1:
CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SENZA FIRMA E SENZA ALLEGATO E HA RICEVUTO QUESTO MESSAGGIO DI ERRORE   HA NOTATO CHE DA QUALCHE GIORNO NON HA PI LA FIRMA IN 

['SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENTI ENGO QUESTA IMMAGINE NELLA EMAIL SIGNATURE  GRAZIE E SALUTI TOBIAI CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAILHALLO \xa0 COULD YOU CREATE ME A OUTLOOK EMAIL SIGNATURE PLEASE THANK YOU TOMASPREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WARENEINGANG SIEMENSSTRAE 11082023  18082023 WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023 CHIUSO PER FERIEGOODS RECEIPT VIA SIEMENS 11082023  18082023 CLOSED FOR HOLIDAY  PER MARTINP ARTHURS PATRICKT VANESSAM SILVIAZ E SAMUELN\xa0 E SOLO DA 020823 AL 110823   BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL MIO ACCOUNT I SEGUENTI ACCESSI  INSERIRE DISCO G NEL MIO ACCOUNT  AGGIUNGERE LA FIRMA ALLA MAIL  AGGIUNGERE IL PROGRAMMA PROCALL AL MIO ACCOUNT GRAZIE E BUON LAVORO LAURA SPAZIAN',
 'CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SENZA FIRMA E SENZA ALLEGATO\xa0E HA RICEVUTO QUESTO MESSAGGIO DI ERRORE\xa0  HA NOTATO CHE DA QUALCHE GIORNO NON HA PI LA FIRMA IN AUT

In [169]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 7):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [170]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENTI ENGO QUESTA IMMAGINE NELLA EMAIL SIGNATURE  GRAZIE E SALUTI TOBIAI CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAILHALLO   COULD YOU CREATE ME A OUTLOOK EMAIL SIGNATURE PLEASE THANK YOU TOMASPREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WARENEINGANG SIEMENSSTRAE 11082023  18082023 WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023 CHIUSO PER FERIEGOODS RECEIPT VIA SIEMENS 11082023  18082023 CLOSED FOR HOLIDAY  PER MARTINP ARTHURS PATRICKT VANESSAM SILVIAZ E SAMUELN  E SOLO DA 020823 AL 110823   BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL MIO ACCOUNT I SEGUENTI ACCESSI  INSERIRE DISCO G NEL MIO ACCOUNT  AGGIUNGERE LA FIRMA ALLA MAIL  AGGIUNGERE IL PROGRAMMA PROCALL AL MIO ACCOUNT GRAZIE E BUON LAVORO LAURA SPAZIAN

Result 1:
CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SENZA FIRMA E SENZA ALLEGATO E HA RICEVUTO QUESTO MESSAGGIO DI ERRORE   HA NOTATO CHE DA QUALCHE GIORNO NON HA PI LA FIRMA IN 

In [171]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENTI ENGO QUESTA IMMAGINE NELLA EMAIL SIGNATURE  GRAZIE E SALUTI TOBIAI CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAILHALLO   COULD YOU CREATE ME A OUTLOOK EMAIL SIGNATURE PLEASE THANK YOU TOMASPREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WARENEINGANG SIEMENSSTRAE 11082023  18082023 WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023 CHIUSO PER FERIEGOODS RECEIPT VIA SIEMENS 11082023  18082023 CLOSED FOR HOLIDAY  PER MARTINP ARTHURS PATRICKT VANESSAM SILVIAZ E SAMUELN  E SOLO DA 020823 AL 110823   BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL MIO ACCOUNT I SEGUENTI ACCESSI  INSERIRE DISCO G NEL MIO ACCOUNT  AGGIUNGERE LA FIRMA ALLA MAIL  AGGIUNGERE IL PROGRAMMA PROCALL AL MIO ACCOUNT GRAZIE E BUON LAVORO LAURA SPAZIAN

Result 1:
CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SENZA FIRMA E SENZA ALLEGATO E HA RICEVUTO QUESTO MESSAGGIO DI ERRORE   HA NOTATO CHE DA QUALCHE GIORNO NON HA PI LA FIRMA IN 

['SALVE SAREBBE DA AGGIUNGERE SU TUTTI GLI UTENTI ENGO QUESTA IMMAGINE NELLA EMAIL SIGNATURE  GRAZIE E SALUTI TOBIAI CANT ADD MY SIGNATURE AUTOMATICALLY ON MY MAILHALLO \xa0 COULD YOU CREATE ME A OUTLOOK EMAIL SIGNATURE PLEASE THANK YOU TOMASPREGO AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WARENEINGANG SIEMENSSTRAE 11082023  18082023 WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023 CHIUSO PER FERIEGOODS RECEIPT VIA SIEMENS 11082023  18082023 CLOSED FOR HOLIDAY  PER MARTINP ARTHURS PATRICKT VANESSAM SILVIAZ E SAMUELN\xa0 E SOLO DA 020823 AL 110823   BUONGIORNO CHIEDO PER FAVORE DI AGGIUNGERE AL MIO ACCOUNT I SEGUENTI ACCESSI  INSERIRE DISCO G NEL MIO ACCOUNT  AGGIUNGERE LA FIRMA ALLA MAIL  AGGIUNGERE IL PROGRAMMA PROCALL AL MIO ACCOUNT GRAZIE E BUON LAVORO LAURA SPAZIAN',
 'CIAO DANIELE MI HA MANDATO DI NUOVO LA MAIL SENZA FIRMA E SENZA ALLEGATO\xa0E HA RICEVUTO QUESTO MESSAGGIO DI ERRORE\xa0  HA NOTATO CHE DA QUALCHE GIORNO NON HA PI LA FIRMA IN AUT

In [172]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [176]:
picklefile = "filtered_df_1_email_signature.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': 'AGGIUNGERE ALLA FIRMA EMAIL IN FONDO IL SEGUENTE TESTO INFO WEGEN FERIEN GESCHLOSSENENTRATA MERCI VIA SIEMENS 11082023  18082023.'}], [{'summary_text': 'CIAO DANIELE, MI SONO ACCORTO, LA MIA FIRMA NELLE, EMAIL SE GUARDO,  GRAZIE MILLECIAO, LUNED PROSSIMA SETTIMANA INIZIA, CI SERVIREBBE PER FAVORE UN NUOVO ACCOUNT CON ACCESSO AGLI SERVER COSI. '}], [{'summary_text': "LA MIA FIRMA EMAIL is a collection of e-mail messages sent between Italy and Italy. It is available in Italian, German, Italian, Spanish and Italian. It's available in 3 versions: 1. VECCHIE POTRESTE PER CORTESIA, 2. ED EVENTUALMENTE CORREGGERLE  GRAZIE, 3. STEFANOIO NELLA FIRMA DELLA MAIL SOTTO HO and 4. "}], [{'summary_text': "Grazie Hanneshell received a new laptop today. She has problems with the e-mail sign-off. She asks Tomas Prego to create her a new one. It's closed for holidays.    It's a holiday. "}], [{'summary_text': 'MI SONO ACCORTO, LA MIA FIRMA, NELLE, SE GUARDO, GRAZIE MILLECIAO, MI POTET

# df_1_email_phishing

In [211]:
import pandas as pd
import re

In [212]:
related_words = ['Phishing', 'Pesca fraudolenta', 'Inganno online', 'Truffa informatica', 'Frode via email',
    'Phishing', 'Online scam', 'Email fraud', 'Deceptive online tactics', 'Cyber deception',
    'Phishing', 'Online-Betrug', 'Internet-Täuschung', 'E-Mail-Betrug', 'Cyber-Betrug',
    'Inganno informatico', 'Pesca di informazioni', 'Truffa digitale',
    'Digital deception', 'Information harvesting', 'Online fraud',
    'Digitale Täuschung', 'Informationsdiebstahl', 'Online-Betrug','Spam', 'Posta indesiderata', 
    'Messaggi non richiesti', 'Posta spazzatura', 'Invio massivo di email',
    'Spam', 'Unsolicited email', 'Junk mail', 'Email overload', 'Bulk messaging',
    'Spam', 'Unerwünschte E-Mails', 'Junk-Mail', 'E-Mail-Überflutung', 'Massenversand von Nachrichten',
    'Posta non desiderata', 'Messaggi di spam', 'Comunicazioni indesiderate',
    'Unwanted mail', 'Spam messages', 'Unsolicited communications',
    'Unerwünschte Post', 'Spam-Nachrichten', 'Ungebetene Kommunikation']

In [213]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_1_email_phishing = df_1_email[df_1_email['Text'].str.contains(pattern, case=False)]
filtered_df_1_email_phishing = filtered_df_1_email_phishing.reset_index(drop=True)

filtered_df_1_email_phishing

,Title,Topic,Text
0,Possibile phishing,"{""16"":""User identity management, Permissions, ...",CIAO DAL CENTRALINO CI HANNO GIRATO QUESTA MA...
1,Phishing,"{""16"":""User identity management, Permissions, ...",CIAO NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PA...
2,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...
3,Problema arrivo ed invio mail,"{""16"":""User identity management, Permissions, ...",CIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLE...
4,Mail Spam,"{""16"":""User identity management, Permissions, ...",CIAO SULLA MAIL FATTURE FORNITORI ARRIVATA QU...
5,SPAM,"{""16"":""User identity management, Permissions, ...",CIAO VI INFORMO CHE MI ARRIVATA LA SEGUENTE M...
6,Spam Mail?,"{""16"":""User identity management, Permissions, ...",CIAO VOLEVO SOLO CHIEDERVI DIRVI CHE IO E ANC...
7,FW: Rechnung B201303056 Service GmbH WKÖ,"{""16"":""User identity management, Permissions, ...",HALLO IST OK SPAM VON MEINEMMEINER GAL...
8,Spam,"{""16"":""User identity management, Permissions, ...",CIAO UN NOSTRO POTENZIALE CLIENTE HA PROVATO ...
9,Phishing,"{""16"":""User identity management, Permissions, ...",CIAO NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PA...


In [214]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_1_email_phishing[i:i+n] for i in range(0,len(filtered_df_1_email_phishing),n)]
    #print(list_df)

In [215]:
len(list_df)

4

In [216]:
list_df[3]

,Title,Topic,Text
15,FW: Rechnung B201303056 Service GmbH WKÖ,"{""16"":""User identity management, Permissions, ...",HALLO IST OK SPAM VON MEINEMMEINER GAL...
16,FW: Rechnung B201303056 Service GmbH WKÖ,"{""16"":""User identity management, Permissions, ...",HALLO IST OK SPAM VON MEINEMMEINER GAL...


In [217]:
def put_together():
    list1=[]
    for index in range(0,3):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
CIAO  DAL CENTRALINO CI HANNO GIRATO QUESTA MAIL SULLA MAIL FATTURE FORNITORI MA MI SEMBRA MOLTO UN TENTATIVO DI PHISHING VEDENDO LINDIRIZZO FIRMATO EMI CONTROL GRAZIECIAO  NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PARTNER CI HANNO CONTATTATO PERCHE HANNO RICEVUTO DELLE EMAIL PHISHING DALLINIDIRZZO PDANIELSSONEMICONTROLSCOM SE VOLETE POSSO GIRARVI LE EMAIL SALUTI PATRICK  HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLEMI RIGUARDO ALLINVIO E ALLARRIVO DEI MAIL SIA INTERNI CHE ESTERNI ECCO GLI ERRORIERRORE IN INTERNO CON MESSAGGIO DI VIOLAZIONE DI CONDIVISIONE DEI DATI SENZA DOCUMENTI ANNESSI ERRORE IN INTERNO ED ESTERNO DI VIOLAZIONE DI CONDIVISIONE DEI DATI CON E SENZA DOCUMENTI ANNESSI ERRORE DI ARRIVO DEI MAIL ANCHE PER ANTOINE DI CARLO E EMMANUEL CHANCEL  MICHEL BLANCHOT HA RISPOSTO AD UNA MIA MAI

['CIAO\xa0 DAL CENTRALINO CI HANNO GIRATO QUESTA MAIL SULLA MAIL FATTURE FORNITORI MA MI SEMBRA MOLTO UN TENTATIVO DI PHISHING VEDENDO LINDIRIZZO FIRMATO EMI CONTROL GRAZIECIAO\xa0 NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PARTNER CI HANNO CONTATTATO PERCHE HANNO RICEVUTO DELLE EMAIL PHISHING DALLINIDIRZZO PDANIELSSONEMICONTROLSCOM SE VOLETE POSSO GIRARVI LE EMAIL SALUTI PATRICK \xa0HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLEMI RIGUARDO ALLINVIO E ALLARRIVO DEI MAIL SIA INTERNI CHE ESTERNI ECCO GLI ERRORIERRORE IN\xa0INTERNO CON MESSAGGIO DI VIOLAZIONE DI CONDIVISIONE DEI DATI SENZA DOCUMENTI ANNESSI ERRORE IN INTERNO ED ESTERNO\xa0DI VIOLAZIONE DI CONDIVISIONE DEI DATI CON E\xa0SENZA DOCUMENTI ANNESSI ERRORE DI ARRIVO DEI MAIL\xa0ANCHE PER ANTOINE DI CARLO E EMMANUEL CHANCEL  MICHEL BLANCHOT HA RISPOSTO A

In [218]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 3):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [219]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
CIAO  DAL CENTRALINO CI HANNO GIRATO QUESTA MAIL SULLA MAIL FATTURE FORNITORI MA MI SEMBRA MOLTO UN TENTATIVO DI PHISHING VEDENDO LINDIRIZZO FIRMATO EMI CONTROL GRAZIECIAO  NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PARTNER CI HANNO CONTATTATO PERCHE HANNO RICEVUTO DELLE EMAIL PHISHING DALLINIDIRZZO PDANIELSSONEMICONTROLSCOM SE VOLETE POSSO GIRARVI LE EMAIL SALUTI PATRICK  HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLEMI RIGUARDO ALLINVIO E ALLARRIVO DEI MAIL SIA INTERNI CHE ESTERNI ECCO GLI ERRORIERRORE IN INTERNO CON MESSAGGIO DI VIOLAZIONE DI CONDIVISIONE DEI DATI SENZA DOCUMENTI ANNESSI ERRORE IN INTERNO ED ESTERNO DI VIOLAZIONE DI CONDIVISIONE DEI DATI CON E SENZA DOCUMENTI ANNESSI ERRORE DI ARRIVO DEI MAIL ANCHE PER ANTOINE DI CARLO E EMMANUEL CHANCEL  MICHEL BLANCHOT HA RISPOSTO AD UNA MIA MAI

In [220]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
CIAO  DAL CENTRALINO CI HANNO GIRATO QUESTA MAIL SULLA MAIL FATTURE FORNITORI MA MI SEMBRA MOLTO UN TENTATIVO DI PHISHING VEDENDO LINDIRIZZO FIRMATO EMI CONTROL GRAZIECIAO  NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PARTNER CI HANNO CONTATTATO PERCHE HANNO RICEVUTO DELLE EMAIL PHISHING DALLINIDIRZZO PDANIELSSONEMICONTROLSCOM SE VOLETE POSSO GIRARVI LE EMAIL SALUTI PATRICK  HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLEMI RIGUARDO ALLINVIO E ALLARRIVO DEI MAIL SIA INTERNI CHE ESTERNI ECCO GLI ERRORIERRORE IN INTERNO CON MESSAGGIO DI VIOLAZIONE DI CONDIVISIONE DEI DATI SENZA DOCUMENTI ANNESSI ERRORE IN INTERNO ED ESTERNO DI VIOLAZIONE DI CONDIVISIONE DEI DATI CON E SENZA DOCUMENTI ANNESSI ERRORE DI ARRIVO DEI MAIL ANCHE PER ANTOINE DI CARLO E EMMANUEL CHANCEL  MICHEL BLANCHOT HA RISPOSTO AD UNA MIA MAI

['CIAO\xa0 DAL CENTRALINO CI HANNO GIRATO QUESTA MAIL SULLA MAIL FATTURE FORNITORI MA MI SEMBRA MOLTO UN TENTATIVO DI PHISHING VEDENDO LINDIRIZZO FIRMATO EMI CONTROL GRAZIECIAO\xa0 NEGLI ULTIMI 2 GIORNI VARI DEI NOSTRI PARTNER CI HANNO CONTATTATO PERCHE HANNO RICEVUTO DELLE EMAIL PHISHING DALLINIDIRZZO PDANIELSSONEMICONTROLSCOM SE VOLETE POSSO GIRARVI LE EMAIL SALUTI PATRICK \xa0HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT LOGIN BECAUSE I FORGOT MY PASSWORD WHEN I TRY TO RESET MY PASSWORD I GET NO EMAIL TO RESET IT ALSO THE SPAM ORDER IS EMPTY GREETINGS IVANCIAO MICHEL BLANCHOT MI HA INFORMATO DI PROBLEMI RIGUARDO ALLINVIO E ALLARRIVO DEI MAIL SIA INTERNI CHE ESTERNI ECCO GLI ERRORIERRORE IN\xa0INTERNO CON MESSAGGIO DI VIOLAZIONE DI CONDIVISIONE DEI DATI SENZA DOCUMENTI ANNESSI ERRORE IN INTERNO ED ESTERNO\xa0DI VIOLAZIONE DI CONDIVISIONE DEI DATI CON E\xa0SENZA DOCUMENTI ANNESSI ERRORE DI ARRIVO DEI MAIL\xa0ANCHE PER ANTOINE DI CARLO E EMMANUEL CHANCEL  MICHEL BLANCHOT HA RISPOSTO A

# df_1_rights

In [49]:
#df_2_rights
import pandas as pd

words_to_check = ['badge', 'accedere', 'diritti', 'access', 'porte', 'mensa', 'accesso']
# Initialize an empty DataFrame to store the matching rows
df_1_rights = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_rights = df_1_rights.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_rights = df_1_rights.reset_index(drop=True)

df_1_rights

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3570219059.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_rights = df_1_rights.appe

,Title,Topic,Text,tokens
0,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIAN...,"[badge, adriana, amodio, dovrebbe, nr, 10147, ..."
1,Reserve Badge,"{""16"":""User identity management, Permissions, ...",CIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA...,"[rinominare, seguenti, badge, kaba, accessi, 8..."
2,Neuer Mitarbeiter,"{""16"":""User identity management, Permissions, ...",HALLOBITTE FR UNSEREN NEUEN MITARBEITER TOBIAS...,"[hallobitte, unseren, tobias, haas, soweit, al..."
3,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...,"[programmare, badge, 10147, segue, accesso, po..."
4,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...,"[badge, gajdos, stessi, accessi, altri, saldat..."
...,...,...,...,...
378,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...,"[mysql, server, esterno, bloccato, taiotmysqld..."
379,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...,"[assegnato, support, dellit, engo, volevo, chi..."
380,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...,"[cortesemente, accesso, lettura, lorenzo, sart..."
381,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."


In [50]:
df_1_rights

,Title,Topic,Text,tokens
0,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIAN...,"[badge, adriana, amodio, dovrebbe, nr, 10147, ..."
1,Reserve Badge,"{""16"":""User identity management, Permissions, ...",CIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA...,"[rinominare, seguenti, badge, kaba, accessi, 8..."
2,Neuer Mitarbeiter,"{""16"":""User identity management, Permissions, ...",HALLOBITTE FR UNSEREN NEUEN MITARBEITER TOBIAS...,"[hallobitte, unseren, tobias, haas, soweit, al..."
3,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...,"[programmare, badge, 10147, segue, accesso, po..."
4,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...,"[badge, gajdos, stessi, accessi, altri, saldat..."
...,...,...,...,...
378,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...,"[mysql, server, esterno, bloccato, taiotmysqld..."
379,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...,"[assegnato, support, dellit, engo, volevo, chi..."
380,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...,"[cortesemente, accesso, lettura, lorenzo, sart..."
381,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."


In [51]:
df_1_rights = df_1_rights.drop(columns='tokens')
df_1_rights

,Title,Topic,Text
0,Badge Adriana Amodio,"{""16"":""User identity management, Permissions, ...",BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIAN...
1,Reserve Badge,"{""16"":""User identity management, Permissions, ...",CIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA...
2,Neuer Mitarbeiter,"{""16"":""User identity management, Permissions, ...",HALLOBITTE FR UNSEREN NEUEN MITARBEITER TOBIAS...
3,attivazione badge n. 10147 per nuova collabora...,"{""16"":""User identity management, Permissions, ...",EGREGI SIGNORI VI PREGO DI PROGRAMMARE IL BADG...
4,Martin Gajdos,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVE UN BADGE PER MARTIN GAJDOS C...
...,...,...,...
378,accesso a taiotmysqldb.mysql.database.azure.co...,"{""16"":""User identity management, Permissions, ...",LACCESSO AL MYSQL SERVER ESTERNO BLOCCATO TAI...
379,IT Admin for Engo,"{""16"":""User identity management, Permissions, ...",CIAO MI HANNO ASSEGNATO IL SUPPORT DELLIT PER...
380,Accesso a cartelle per Lorenzo Sartori,"{""16"":""User identity management, Permissions, ...",BUONGIORNO CORTESEMENTE POSSIBILE DARE ACCESS...
381,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...


# df_1_computer

In [52]:
#df_3_computer
import pandas as pd

words_to_check = ['pc', 'laptop', 'computer', 'phone', 'password']
# Initialize an empty DataFrame to store the matching rows
df_1_computer = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_computer = df_1_computer.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_computer = df_1_computer.reset_index(drop=True)

df_1_computer

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df_1_computer.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3462105897.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_computer = df

,Title,Topic,Text,tokens
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...,"[penso, yu, yang, cambiato, password, tempo, e..."
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...,"[ciaomi, mia, signatura, nelloutlook, dopo, ca..."
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...,"[ragazzi, vorrei, accedere, rete, intranet, vi..."
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...,"[access, sapconcur, cant, login, because, forg..."
...,...,...,...,...
127,Myprotime account,"{""16"":""User identity management, Permissions, ...",NON RIESCO AD ACCEDERE AL MIO ACCOUNT MYPROTIM...,"[accedere, account, myprotime, password, usata..."
128,E-Mail nicht mehr aktiv,"{""16"":""User identity management, Permissions, ...",OLD PASSWORD ALPIN123,"[old, password, alpin123]"
129,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."
130,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...,"[servirebbe, vostro, admin, password, poter, i..."


In [53]:
df_1_computer

,Title,Topic,Text,tokens
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...,"[penso, yu, yang, cambiato, password, tempo, e..."
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...,"[ciaomi, mia, signatura, nelloutlook, dopo, ca..."
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...,"[ragazzi, vorrei, accedere, rete, intranet, vi..."
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...,"[access, sapconcur, cant, login, because, forg..."
...,...,...,...,...
127,Myprotime account,"{""16"":""User identity management, Permissions, ...",NON RIESCO AD ACCEDERE AL MIO ACCOUNT MYPROTIM...,"[accedere, account, myprotime, password, usata..."
128,E-Mail nicht mehr aktiv,"{""16"":""User identity management, Permissions, ...",OLD PASSWORD ALPIN123,"[old, password, alpin123]"
129,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...,"[nell, ufficio, denver, c, pc, alyssa, sempre,..."
130,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...,"[servirebbe, vostro, admin, password, poter, i..."


In [54]:
df_1_computer = df_1_computer.drop(columns='tokens')
df_1_computer

,Title,Topic,Text
0,PASSWORD WINDOWS bloccato,"{""16"":""User identity management, Permissions, ...",CIAO PENSO CHE LE YU YANG NON HA CAMBIATO LA P...
1,Signatura,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE AGGIUNGERE MIA SIGNATURA NELLOUT...
2,Intranet/Protime/abilitazione badge per produz...,"{""16"":""User identity management, Permissions, ...",CIAO RAGAZZI 1 VORREI ACCEDERE ALLA RETE INTRA...
3,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...
4,SAP Concur,"{""16"":""User identity management, Permissions, ...",HELLO I NEED ACCESS TO MY SAPCONCUR BUT I CANT...
...,...,...,...
127,Myprotime account,"{""16"":""User identity management, Permissions, ...",NON RIESCO AD ACCEDERE AL MIO ACCOUNT MYPROTIM...
128,E-Mail nicht mehr aktiv,"{""16"":""User identity management, Permissions, ...",OLD PASSWORD ALPIN123
129,PC Alyssa Klotz,"{""16"":""User identity management, Permissions, ...",CIAO NELL UFFICIO A DENVER C ANCORA IL PC DI A...
130,Radix,"{""16"":""User identity management, Permissions, ...",CIAO MI SERVIREBBE IL VOSTRO ADMIN PASSWORD PE...


# df_1_software

In [55]:
#df_1_software
import pandas as pd

words_to_check = ['server', 'nav', 'vpn', 'crm', 'sap', 'concur']
# Initialize an empty DataFrame to store the matching rows
df_1_software = pd.DataFrame(columns=df_1.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_1['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_1_software = df_1_software.append(df_1[word_mask])

# Reset the index of the new DataFrame
df_1_software = df_1_software.reset_index(drop=True)

df_1_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_software.append(df_1[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\369636750.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_1_software = df_1_s

,Title,Topic,Text,tokens
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...,"[koennen, lserver, installieren, fuer, neue, k..."
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...,"[aber, koennen, server, finden, offnen]"
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...,"[contattato, stephane, detto, dato, diritti, q..."
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...,"[dopo, cambiamento, server, mandare, email, da..."
...,...,...,...,...
165,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...,"[impostare, lutente, andrej, mladenoski, utent..."
166,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...,"[dear, all, create, concur, account, employee,..."
167,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...,"[conto, sap, concur, stato, creato, antoine, p..."
168,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...,"[noted, access, sap, concur, our, newly, emplo..."


In [56]:
df_1_software

,Title,Topic,Text,tokens
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...,"[koennen, lserver, installieren, fuer, neue, k..."
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...,"[aber, koennen, server, finden, offnen]"
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...,"[pc, cant, connect, servers, tafr, et, tab]"
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...,"[contattato, stephane, detto, dato, diritti, q..."
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...,"[dopo, cambiamento, server, mandare, email, da..."
...,...,...,...,...
165,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...,"[impostare, lutente, andrej, mladenoski, utent..."
166,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...,"[dear, all, create, concur, account, employee,..."
167,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...,"[conto, sap, concur, stato, creato, antoine, p..."
168,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...,"[noted, access, sap, concur, our, newly, emplo..."


In [57]:
df_1_software = df_1_software.drop(columns='tokens')
df_1_software

,Title,Topic,Text
0,L-server,"{""16"":""User identity management, Permissions, ...",HALLO KOENNEN SIE BITTE LSERVER INSTALLIEREN ...
1,L-server,"{""16"":""User identity management, Permissions, ...",OK ABER WIR KOENNEN NICHT SERVER FINDEN UND OF...
2,New PC - Pb connexions to Servers,"{""16"":""User identity management, Permissions, ...",I HAVE A NEW PC BUT I CANT CONNECT SERVERS TAF...
3,Permission disk,"{""16"":""User identity management, Permissions, ...",CIAO HO CONTATTATO STEPHANE E MI HA DETTO CHE ...
4,Uscita mail info@epowertrac.com,"{""16"":""User identity management, Permissions, ...",DOPO IL CAMBIAMENTO SERVER SU TA NON PI POSSI...
...,...,...,...
165,Concur Andrej Mladenoski,"{""16"":""User identity management, Permissions, ...",CIAO VI PREGO DI IMPOSTARE LUTENTE ANDREJ MLA...
166,Concur,"{""16"":""User identity management, Permissions, ...",DEAR ALL COULD YOU CREATE THE CONCUR ACCOUNT F...
167,Conto user ANTOINE PAULY,"{""16"":""User identity management, Permissions, ...",CIAO IL CONTO SAP CONCUR STATO CREATO PER ANT...
168,SAP Concur for Patrick Schgaguler,"{""16"":""User identity management, Permissions, ...",HI WE NOTED THAT THE ACCESS FOR SAP CONCUR FOR...


# df_2
{"12":"Standard Software: Operating systems, Microsoft Office, Citrix clients, CRM Plugin, Acrobat Reader etc."}

In [58]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella', 'problem', 'poi', 'digitale','also','du', 'problema', 'io', 
                    'es', 'here', 'pdf', 'devo', 'as','marco','41', 'provato', 'install', 
                    'tutto', 'eine','va', 'sempre', 'open','work','do','programma','one']

# Tokenize and remove stopwords from the 'Text' column
df_2['tokens'] = df_2['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_2['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('email', 58), ('pc', 56), ('outlook', 54), ('teams', 43), ('installare', 35), ('pw', 34), ('nav', 33), ('mail', 33), ('laptop', 30), ('password', 29), ('computer', 27), ('office', 25), ('accedere', 24), ('accesso', 20), ('message', 20), ('acrobat', 19), ('vpn', 19), ('prfprotokoll', 19), ('passwort', 18), ('vdi', 18), ('server', 18), ('citrix', 18), ('microsoft', 17), ('edge', 16), ('adobe', 15), ('excel', 15), ('cant', 15), ('vedo', 15), ('app', 15), ('emails', 15)]


# df_2_pc

In [59]:
#df_2_pc
import pandas as pd

words_to_check = ['pc', 'computer', 'accedere', 'pw', 'password',
                  'laptop', 'passwort', 'pwd', 'login', 'accesso']

# Initialize an empty DataFrame to store the matching rows
df_2_pc = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_pc = df_2_pc.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_pc = df_2_pc.reset_index(drop=True)

df_2_pc

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3194719117.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_pc = df_2_pc.append(df_2[word_mask])
C:\Users\Mat

,Title,Topic,Text,tokens
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...,"[computer, c405, teams, persona, chiama, suona..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...,"[nach, einem, neustart, meines, pc, scheint, p..."
3,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...,"[pc, nav, italie, austria, doesnt, best]"
4,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...,"[servirebbe, linstallazione, windows, 10, pc, ..."
...,...,...,...,...
272,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...,"[limpostazione, indicata, riesce, cortesemente..."
273,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
274,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
275,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."


In [60]:
df_2_pc

,Title,Topic,Text,tokens
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...,"[computer, c405, teams, persona, chiama, suona..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...,"[nach, einem, neustart, meines, pc, scheint, p..."
3,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...,"[pc, nav, italie, austria, doesnt, best]"
4,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...,"[servirebbe, linstallazione, windows, 10, pc, ..."
...,...,...,...,...
272,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...,"[limpostazione, indicata, riesce, cortesemente..."
273,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
274,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
275,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."


In [61]:
df_2_pc = df_2_pc.drop(columns='tokens')
df_2_pc

,Title,Topic,Text
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...
2,Probleme mit Procall,"{""12"":""Standard Software: Operating systems, M...",HALLO AUCH NACH EINEM NEUSTART MEINES PC SCHEI...
3,Navision,"{""12"":""Standard Software: Operating systems, M...",HI I HAVE A NEW PC NAV FR AND ITALIE AND AUSTR...
4,Update a Windows 10 urgente (Win 7 installato),"{""12"":""Standard Software: Operating systems, M...",SERVIREBBE LINSTALLAZIONE DI WINDOWS 10 SU UN ...
...,...,...,...
272,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...
273,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...
274,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...
275,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...


# df_2_email

In [62]:
#df_2_email
import pandas as pd

words_to_check = ['message', 'email', 'outlook', 'emails', 
                  'mail', 'accedere', 'pw', 'password', 'passwort', 
                  'recipient', 'attachments', 'pwd', 'microsoft', 'office', 'app', 'login', 'accesso']


# Initialize an empty DataFrame to store the matching rows
df_2_email = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_email = df_2_email.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_email = df_2_email.reset_index(drop=True)

df_2_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\698636073.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_email = df_2_email.append(df_2[word

,Title,Topic,Text,tokens
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...,"[absender, whitelist, setzen, wichtig, fehlerm..."
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...,"[press, cirtix, remote, desktop, symbol, conne..."
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...,"[ultimamente, abbiamo, problemi, grossi, mail,..."
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...,"[helloafter, changed, password, cant, connect,..."
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...,"[linda, zhang, accessing, italian, system, cit..."
...,...,...,...,...
464,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...,"[limpostazione, indicata, riesce, cortesemente..."
465,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
466,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
467,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."


In [63]:
df_2_email

,Title,Topic,Text,tokens
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...,"[absender, whitelist, setzen, wichtig, fehlerm..."
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...,"[press, cirtix, remote, desktop, symbol, conne..."
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...,"[ultimamente, abbiamo, problemi, grossi, mail,..."
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...,"[helloafter, changed, password, cant, connect,..."
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...,"[linda, zhang, accessing, italian, system, cit..."
...,...,...,...,...
464,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...,"[limpostazione, indicata, riesce, cortesemente..."
465,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
466,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...,"[stesso, segnalato, risolto, gennaio, febbraio..."
467,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...,"[necessito, configurare, laptop, allaccesso, t..."


In [64]:
df_2_email = df_2_email.drop(columns = 'tokens')
df_2_email

,Title,Topic,Text
0,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...
1,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...
2,WG: Non remis : FW: FR0042-23-06A POA Tête dou...,"{""12"":""Standard Software: Operating systems, M...",ULTIMAMENTE ABBIAMO DEI PROBLEMI GROSSI CON L...
3,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...
4,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...
...,...,...,...
464,Accesso Docfinance,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO LIMPOSTAZIONE INDICATA NON MI RIESC...
465,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...
466,Accesso servse licenza Codewarrior,"{""12"":""Standard Software: Operating systems, M...",STESSO PROBLEMA SEGNALATO E RISOLTO A GENNAIO ...
467,Accesso VPN e verifica collegamenti disco L,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NECESSITO DI CONFIGURARE IL MIO LAP...


# Further refinement

## look for df_2_email 'access'

In [65]:
import pandas as pd
import re

In [66]:
related_words = ['access', 'accessing', 'accessible', 'accessed', 'accesses', 'entry', 
                 'approach', 'admission', 'entrance', 'reach', 'open', 'connect', 
                 'visit', 'retrieve', 'use','Zugang', 'Zugriff', 'zugänglich',
                 'zugriffen', 'Zugänge', 'Eingang', 'Herangehensweise', 'Zulassung',
                 'Eingang', 'erreichen', 'öffnen', 'verbinden', 'besuchen', 'abrufen', 'benutzen',
                 'accesso', 'accesso', 'accessibile', 'accesso', 'accessi', 
                 'entrata', 'approccio', 'ammissione', 'ingresso', 'raggiungere', 
                 'aprire', 'collegare', 'visitare', 'recuperare', 'usare']

In [67]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_2_email = df_2_email[df_2_email['Text'].str.contains(pattern, case=False)]
filtered_df_2_email = filtered_df_2_email.reset_index(drop=True)

filtered_df_2_email

,Title,Topic,Text
0,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...
1,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...
2,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...
3,can't open contracts in d.3 smart explorer,"{""12"":""Standard Software: Operating systems, M...",HI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY ...
4,Activate Programs/Licences,"{""12"":""Standard Software: Operating systems, M...",HI EVERYONE I WOULD NEED THE EMAIL SIGNATURE M...
...,...,...,...
87,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",POSSIAMO ANCHE AGGIUNGERE ACCESSO AL DRIVE XMO...
88,Accesso Folder,"{""12"":""Standard Software: Operating systems, M...",ACCESSO FOLDER
89,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",CIAO STO PROVANDO DI LAVORARE CON CITRIX MA NO...
90,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",TUTTI I DOCUMENTI CHE HO SALVATO IN LOCALE SUL...


In [68]:
filtered_df_2_email

,Title,Topic,Text
0,Virtual Machine can`t be started,"{""12"":""Standard Software: Operating systems, M...",HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP ...
1,Outlook issue,"{""12"":""Standard Software: Operating systems, M...",HELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ...
2,Linda Zhang - Excel is not working over Citrix...,"{""12"":""Standard Software: Operating systems, M...",CIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN...
3,can't open contracts in d.3 smart explorer,"{""12"":""Standard Software: Operating systems, M...",HI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY ...
4,Activate Programs/Licences,"{""12"":""Standard Software: Operating systems, M...",HI EVERYONE I WOULD NEED THE EMAIL SIGNATURE M...
...,...,...,...
87,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",POSSIAMO ANCHE AGGIUNGERE ACCESSO AL DRIVE XMO...
88,Accesso Folder,"{""12"":""Standard Software: Operating systems, M...",ACCESSO FOLDER
89,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",CIAO STO PROVANDO DI LAVORARE CON CITRIX MA NO...
90,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",TUTTI I DOCUMENTI CHE HO SALVATO IN LOCALE SUL...


In [69]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_2_email[i:i+n] for i in range(0,len(filtered_df_2_email),n)]
    #print(list_df)

In [70]:
len(list_df)

19

In [71]:
list_df[18]

,Title,Topic,Text
90,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",TUTTI I DOCUMENTI CHE HO SALVATO IN LOCALE SUL...
91,Citrix non va?,"{""12"":""Standard Software: Operating systems, M...",TUTTI I DOCUMENTI CHE HO SALVATO IN LOCALE SUL...


In [72]:
def put_together():
    list1=[]
    for index in range(0,18):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [73]:
summ_block = put_together()
summ_block

Result 0:
HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP SYMBOL TO CONNECT TO MY VIRTUAL MACHINE THIS MESSAGE SHOWS UP AN THE VIRTUAL MACHINE DOESNT OPENS I CANT REALLY WORK WITHOUT THE VIRTUAL MACHINE SO PLEASE TRY TO FIX IT SOONHELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ME TO OUTLOOK I HAVE AN ERROR MESSAGE COULD YOU DO SOMETHING PLEASE  BEST REGARDS VINCENT GADOISCIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN SYSTEM VIA CITRIX VDI DESKTOP SHE HAS TO DOWNLOAD EXCEL LISTS FROM NAV EDIT AND LINK THEM TO NAV CLAIMSWHEN SHE TRIES TO OPEN AND EDIT THE EXCEL LIST IT LOGS ON WITH HER USER LINDA ZHANG BUT AN ERROR MESSAGE APPEARS AND SAYS THAT WHITH THIS ACCOUNT IT IS NOT POSSIBLE TO USE EXCEL IN VDI IS THIS A LICENSING PROBLEM BR MARTINHI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY TO OPEN AN CONTRACT DXIG THIS MESSAGE IS DISPLAYED IT SAYS I SHOULD ACTIVATE D3 VIEW VERWENDEN BUT IT IS ACTIVATED CAN YOU HELP ME HERE PLEASEHI EVERYONE I WOULD NEED THE EMAIL SIGNATURE MANAGER AND ADOBE ACROBA

['HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP SYMBOL TO CONNECT TO MY VIRTUAL MACHINE THIS MESSAGE SHOWS UP AN THE VIRTUAL MACHINE DOESNT OPENS I CANT REALLY WORK WITHOUT THE VIRTUAL MACHINE SO PLEASE TRY TO FIX IT SOONHELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ME TO OUTLOOK I HAVE AN ERROR MESSAGE COULD YOU DO SOMETHING PLEASE  BEST REGARDS VINCENT GADOISCIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN SYSTEM VIA CITRIX VDI DESKTOP SHE HAS TO DOWNLOAD EXCEL LISTS FROM NAV EDIT AND LINK THEM TO NAV CLAIMSWHEN SHE TRIES TO OPEN AND EDIT THE EXCEL LIST IT LOGS ON WITH HER USER LINDA ZHANG BUT AN ERROR MESSAGE APPEARS AND SAYS THAT WHITH THIS ACCOUNT IT IS NOT POSSIBLE TO USE EXCEL IN VDI IS THIS A LICENSING PROBLEM BR MARTINHI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY TO OPEN AN CONTRACT DXIG THIS MESSAGE IS DISPLAYED IT SAYS I SHOULD ACTIVATE\xa0D3 VIEW VERWENDEN BUT IT IS ACTIVATED CAN YOU HELP ME HERE PLEASEHI EVERYONE I\xa0WOULD NEED THE EMAIL SIGNATURE MANAGER AND ADOBE ACROBAT 

In [74]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 18):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [75]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP SYMBOL TO CONNECT TO MY VIRTUAL MACHINE THIS MESSAGE SHOWS UP AN THE VIRTUAL MACHINE DOESNT OPENS I CANT REALLY WORK WITHOUT THE VIRTUAL MACHINE SO PLEASE TRY TO FIX IT SOONHELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ME TO OUTLOOK I HAVE AN ERROR MESSAGE COULD YOU DO SOMETHING PLEASE  BEST REGARDS VINCENT GADOISCIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN SYSTEM VIA CITRIX VDI DESKTOP SHE HAS TO DOWNLOAD EXCEL LISTS FROM NAV EDIT AND LINK THEM TO NAV CLAIMSWHEN SHE TRIES TO OPEN AND EDIT THE EXCEL LIST IT LOGS ON WITH HER USER LINDA ZHANG BUT AN ERROR MESSAGE APPEARS AND SAYS THAT WHITH THIS ACCOUNT IT IS NOT POSSIBLE TO USE EXCEL IN VDI IS THIS A LICENSING PROBLEM BR MARTINHI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY TO OPEN AN CONTRACT DXIG THIS MESSAGE IS DISPLAYED IT SAYS I SHOULD ACTIVATE D3 VIEW VERWENDEN BUT IT IS ACTIVATED CAN YOU HELP ME HERE PLEASEHI EVERYONE I WOULD NEED THE EMAIL SIGNATURE MANAGER AND ADOBE ACROBA

In [76]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP SYMBOL TO CONNECT TO MY VIRTUAL MACHINE THIS MESSAGE SHOWS UP AN THE VIRTUAL MACHINE DOESNT OPENS I CANT REALLY WORK WITHOUT THE VIRTUAL MACHINE SO PLEASE TRY TO FIX IT SOONHELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ME TO OUTLOOK I HAVE AN ERROR MESSAGE COULD YOU DO SOMETHING PLEASE  BEST REGARDS VINCENT GADOISCIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN SYSTEM VIA CITRIX VDI DESKTOP SHE HAS TO DOWNLOAD EXCEL LISTS FROM NAV EDIT AND LINK THEM TO NAV CLAIMSWHEN SHE TRIES TO OPEN AND EDIT THE EXCEL LIST IT LOGS ON WITH HER USER LINDA ZHANG BUT AN ERROR MESSAGE APPEARS AND SAYS THAT WHITH THIS ACCOUNT IT IS NOT POSSIBLE TO USE EXCEL IN VDI IS THIS A LICENSING PROBLEM BR MARTINHI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY TO OPEN AN CONTRACT DXIG THIS MESSAGE IS DISPLAYED IT SAYS I SHOULD ACTIVATE D3 VIEW VERWENDEN BUT IT IS ACTIVATED CAN YOU HELP ME HERE PLEASEHI EVERYONE I WOULD NEED THE EMAIL SIGNATURE MANAGER AND ADOBE ACROBA

['HELLO IF I PRESS ON THE CIRTIX REMOTE DESKTOP SYMBOL TO CONNECT TO MY VIRTUAL MACHINE THIS MESSAGE SHOWS UP AN THE VIRTUAL MACHINE DOESNT OPENS I CANT REALLY WORK WITHOUT THE VIRTUAL MACHINE SO PLEASE TRY TO FIX IT SOONHELLOAFTER CHANGED MY PASSWORD I CANT CONNECT ME TO OUTLOOK I HAVE AN ERROR MESSAGE COULD YOU DO SOMETHING PLEASE  BEST REGARDS VINCENT GADOISCIAO WHEN LINDA ZHANG IS ACCESSING THE ITALIAN SYSTEM VIA CITRIX VDI DESKTOP SHE HAS TO DOWNLOAD EXCEL LISTS FROM NAV EDIT AND LINK THEM TO NAV CLAIMSWHEN SHE TRIES TO OPEN AND EDIT THE EXCEL LIST IT LOGS ON WITH HER USER LINDA ZHANG BUT AN ERROR MESSAGE APPEARS AND SAYS THAT WHITH THIS ACCOUNT IT IS NOT POSSIBLE TO USE EXCEL IN VDI IS THIS A LICENSING PROBLEM BR MARTINHI GUYS I HAVE A PROBLEM WITH MY DMS IF I TRY TO OPEN AN CONTRACT DXIG THIS MESSAGE IS DISPLAYED IT SAYS I SHOULD ACTIVATE\xa0D3 VIEW VERWENDEN BUT IT IS ACTIVATED CAN YOU HELP ME HERE PLEASEHI EVERYONE I\xa0WOULD NEED THE EMAIL SIGNATURE MANAGER AND ADOBE ACROBAT 

In [77]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [79]:
import pickle

In [81]:
picklefile = "filtered_df_2_email.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': " LINDA ZHANG can't connect to her VIRTUAL MACHINE without the CIRTIX REMOTE DESKTOP SYMBOL. She needs an e-mail manager and an abroobat pro to fix the problem. BRINNIHI has a problem with his DMS."}], [{'summary_text': "HII can't get by the outlook to open on his computer. He has problems with VPN and the CITRIX Inlogger. He can't create a ticket. He leaves his computer with Team Viewer open so he can send e-mails. "}], [{'summary_text': "The TV couldn't reach me via email for three days, but it now works via my hotmail address. My e-mail address is 1 594 117 172 and I have a team viewer password BJYUQGIJ thanks to Pony Express. "}], [{'summary_text': "After changing my password, I can't connect to OutLOOK. I have an error message. I am 1 594 117 172 with the team viewer password BJYUQGIJ and I am having a problem with EMAILSCAN. I will log on to my computer to see what's happening."}], [{'summary_text': "Thanks to a gift from Pony Express, she can't open her e-mail

# df_2_email_signature

In [178]:
import pandas as pd
import re

In [179]:
related_words = [
    'Firma', 'Autografo', 'Sottoscrizione', 'Segno distintivo', 'Sigla', 'Marcatura',
    'Ratifica', 'Apposizione di firma', 'Sigillo', 'Atto autografo',
    'Signature', 'Autograph', 'Mark', 'Seal', 'Sign-off',
    'Inked mark', 'Signatory', 'Handwriting', 'Personal mark',
    'Unterschrift', 'Autogramm', 'Kennzeichen', 'Siegel', 'Namenszug', 'Paraphe',
    'Besiegelung', 'Unterzeichnung', 'Handschrift', 'Eigenhändige Unterschrift'
]

In [180]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_2_email_signature = df_2_email[df_1_email['Text'].str.contains(pattern, case=False)]
filtered_df_2_email_signature = filtered_df_2_email_signature.reset_index(drop=True)

filtered_df_2_email_signature

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2694673618.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_df_2_email_signature = df_2_email[df_1_email['Text'].str.contains(pattern, case=False)]


,Title,Topic,Text
0,E-Mails engo Nord America,"{""12"":""Standard Software: Operating systems, M...",HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUN...
1,Email to TA CN,"{""12"":""Standard Software: Operating systems, M...",HO PROVATO A MANDARLI SINGOLARMENTE MA CON IO ...
2,VPN and antivirus,"{""12"":""Standard Software: Operating systems, M...",DA DANIEL PETTERSSON INVIATO MARTED 9 MAGGIO...
3,Daniel Peckham Computer,"{""12"":""Standard Software: Operating systems, M...",HI DANIEL IS A NEW EMPLOYEE AND NEEDS HIS EMA...
4,spam filter,"{""12"":""Standard Software: Operating systems, M...",BITTE EINSTELLEN DASS EMAILS VON DIESER DOMAIN...
5,outlook cannot log in,"{""12"":""Standard Software: Operating systems, M...",OUTLOOK ALWAYS SHOWN NEED PASSWORD AND SHOWN C...
6,Firma - Outlook,"{""12"":""Standard Software: Operating systems, M...",CIAO IN OUTLOOK NON MI METTE PI LA FIRMA MA N...
7,Outlook on computer,"{""12"":""Standard Software: Operating systems, M...",HII CANT GET BY MY OUTLOOK TO OPEN ON MY COMPU...
8,Outlook,"{""12"":""Standard Software: Operating systems, M...",LACCOUNT OUTLOOK DI SOFIA SI CHIAMA SOFIABET...
9,WG: Fehlermeldung,"{""12"":""Standard Software: Operating systems, M...",BITTE DEN ABSENDER AUF DIE WHITELIST SETZEN W...


In [181]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_2_email_signature[i:i+n] for i in range(0,len(filtered_df_2_email_signature),n)]
    #print(list_df)

In [182]:
len(list_df)

8

In [183]:
list_df[7]

,Title,Topic,Text
35,NT Dynamics NAV 2017,"{""12"":""Standard Software: Operating systems, M...",HELLO PLEASE CAN YOU HELP ME TO UPDATE NT DYNA...


In [184]:
def put_together():
    list1=[]
    for index in range(0,7):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUNG ERHALTEN DASS SMTLICHE EMAILS ACCOUNTS UNSERER FILIALE IN CANADA NICHT FUNKTIONIEREN DIE EMAIL ACCOUNTS SIND FOLGENDE DAVIDSTEWARTENGOICECOM  DAVID HAT ZB TELEFONATE ERHALTEN DASS DIE AN IHN GESENDETEN EMAILS MIT DER RCKANTWORT ZURCKKAMEN DASS SIE AN DEN EMPFNGER NICHT ZUGESTELLT WERDEN KONNTEN STEVEDAVEENGOICECOM TODDLANGERENGOICECOM  DA ES SICH UM OFFIZIELLE EMAILS ACCOUNTS HANDELT AUCH FR DIE EINGABE VON ANGEBOTEN IN BEZUG AUF AUSSCHREIBUNGEN BITTE ICH EUCH DAS PROBLEM SO SCHNELL WIE MGLICH ZU BEHEBEN  VIELEN DANK FR DIE SCHNELLE ERLEDIGUNG UND MIT BESTEN GREN MARKUS PROFANTERHO PROVATO A MANDARLI SINGOLARMENTE MA CON IO SEGUENTI EMAIL HO RICEVUTO IL MESSAGGIO DERRORE SUNNYSHENTECHNOALPINCOM DAISYZHAOTECHNOALPINCOM MIAXIONGTECHNOALPINCOM FLORIANHAJZERITECHNOALPINCOM DA DANIEL PETTERSSON  INVIATO MARTED 9 MAGGIO 2023 1133 A DANIELE NERI  OGGETTO VPN AND ANTIVIRUS     HELLO   I HAD PROBLEM WITH VPN AND THE CITRIX INLOGG NOW I C

['HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUNG ERHALTEN DASS SMTLICHE EMAILS ACCOUNTS UNSERER FILIALE IN CANADA NICHT FUNKTIONIEREN DIE EMAIL ACCOUNTS SIND FOLGENDE DAVIDSTEWARTENGOICECOM\xa0 DAVID HAT ZB TELEFONATE ERHALTEN DASS DIE AN IHN GESENDETEN EMAILS MIT DER RCKANTWORT ZURCKKAMEN DASS SIE AN DEN EMPFNGER NICHT ZUGESTELLT WERDEN KONNTEN STEVEDAVEENGOICECOM TODDLANGERENGOICECOM\xa0 DA ES SICH UM OFFIZIELLE EMAILS ACCOUNTS HANDELT AUCH FR DIE EINGABE VON ANGEBOTEN IN BEZUG AUF AUSSCHREIBUNGEN BITTE ICH EUCH DAS PROBLEM SO SCHNELL WIE MGLICH ZU BEHEBEN\xa0 VIELEN DANK FR DIE SCHNELLE ERLEDIGUNG UND MIT BESTEN GREN MARKUS PROFANTERHO PROVATO A MANDARLI SINGOLARMENTE MA CON IO SEGUENTI EMAIL HO RICEVUTO IL MESSAGGIO DERRORE SUNNYSHENTECHNOALPINCOM\xa0DAISYZHAOTECHNOALPINCOM\xa0MIAXIONGTECHNOALPINCOM\xa0FLORIANHAJZERITECHNOALPINCOM DA DANIEL PETTERSSON  INVIATO MARTED 9 MAGGIO 2023 1133 A DANIELE NERI  OGGETTO VPN AND ANTIVIRUS   \xa0 HELLO \xa0 I HAD PROBLEM WITH VPN AND THE CITRI

In [185]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 7):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [186]:
blocks_to_skip = [1]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUNG ERHALTEN DASS SMTLICHE EMAILS ACCOUNTS UNSERER FILIALE IN CANADA NICHT FUNKTIONIEREN DIE EMAIL ACCOUNTS SIND FOLGENDE DAVIDSTEWARTENGOICECOM  DAVID HAT ZB TELEFONATE ERHALTEN DASS DIE AN IHN GESENDETEN EMAILS MIT DER RCKANTWORT ZURCKKAMEN DASS SIE AN DEN EMPFNGER NICHT ZUGESTELLT WERDEN KONNTEN STEVEDAVEENGOICECOM TODDLANGERENGOICECOM  DA ES SICH UM OFFIZIELLE EMAILS ACCOUNTS HANDELT AUCH FR DIE EINGABE VON ANGEBOTEN IN BEZUG AUF AUSSCHREIBUNGEN BITTE ICH EUCH DAS PROBLEM SO SCHNELL WIE MGLICH ZU BEHEBEN  VIELEN DANK FR DIE SCHNELLE ERLEDIGUNG UND MIT BESTEN GREN MARKUS PROFANTERHO PROVATO A MANDARLI SINGOLARMENTE MA CON IO SEGUENTI EMAIL HO RICEVUTO IL MESSAGGIO DERRORE SUNNYSHENTECHNOALPINCOM DAISYZHAOTECHNOALPINCOM MIAXIONGTECHNOALPINCOM FLORIANHAJZERITECHNOALPINCOM DA DANIEL PETTERSSON  INVIATO MARTED 9 MAGGIO 2023 1133 A DANIELE NERI  OGGETTO VPN AND ANTIVIRUS     HELLO   I HAD PROBLEM WITH VPN AND THE CITRIX INLOGG NOW I C

In [187]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUNG ERHALTEN DASS SMTLICHE EMAILS ACCOUNTS UNSERER FILIALE IN CANADA NICHT FUNKTIONIEREN DIE EMAIL ACCOUNTS SIND FOLGENDE DAVIDSTEWARTENGOICECOM  DAVID HAT ZB TELEFONATE ERHALTEN DASS DIE AN IHN GESENDETEN EMAILS MIT DER RCKANTWORT ZURCKKAMEN DASS SIE AN DEN EMPFNGER NICHT ZUGESTELLT WERDEN KONNTEN STEVEDAVEENGOICECOM TODDLANGERENGOICECOM  DA ES SICH UM OFFIZIELLE EMAILS ACCOUNTS HANDELT AUCH FR DIE EINGABE VON ANGEBOTEN IN BEZUG AUF AUSSCHREIBUNGEN BITTE ICH EUCH DAS PROBLEM SO SCHNELL WIE MGLICH ZU BEHEBEN  VIELEN DANK FR DIE SCHNELLE ERLEDIGUNG UND MIT BESTEN GREN MARKUS PROFANTERHO PROVATO A MANDARLI SINGOLARMENTE MA CON IO SEGUENTI EMAIL HO RICEVUTO IL MESSAGGIO DERRORE SUNNYSHENTECHNOALPINCOM DAISYZHAOTECHNOALPINCOM MIAXIONGTECHNOALPINCOM FLORIANHAJZERITECHNOALPINCOM DA DANIEL PETTERSSON  INVIATO MARTED 9 MAGGIO 2023 1133 A DANIELE NERI  OGGETTO VPN AND ANTIVIRUS     HELLO   I HAD PROBLEM WITH VPN AND THE CITRIX INLOGG NOW I C

['HALLO ZUSAMMEN WIR HABEN GESTERN DIE MITTEILUNG ERHALTEN DASS SMTLICHE EMAILS ACCOUNTS UNSERER FILIALE IN CANADA NICHT FUNKTIONIEREN DIE EMAIL ACCOUNTS SIND FOLGENDE DAVIDSTEWARTENGOICECOM\xa0 DAVID HAT ZB TELEFONATE ERHALTEN DASS DIE AN IHN GESENDETEN EMAILS MIT DER RCKANTWORT ZURCKKAMEN DASS SIE AN DEN EMPFNGER NICHT ZUGESTELLT WERDEN KONNTEN STEVEDAVEENGOICECOM TODDLANGERENGOICECOM\xa0 DA ES SICH UM OFFIZIELLE EMAILS ACCOUNTS HANDELT AUCH FR DIE EINGABE VON ANGEBOTEN IN BEZUG AUF AUSSCHREIBUNGEN BITTE ICH EUCH DAS PROBLEM SO SCHNELL WIE MGLICH ZU BEHEBEN\xa0 VIELEN DANK FR DIE SCHNELLE ERLEDIGUNG UND MIT BESTEN GREN MARKUS PROFANTERHO PROVATO A MANDARLI SINGOLARMENTE MA CON IO SEGUENTI EMAIL HO RICEVUTO IL MESSAGGIO DERRORE SUNNYSHENTECHNOALPINCOM\xa0DAISYZHAOTECHNOALPINCOM\xa0MIAXIONGTECHNOALPINCOM\xa0FLORIANHAJZERITECHNOALPINCOM DA DANIEL PETTERSSON  INVIATO MARTED 9 MAGGIO 2023 1133 A DANIELE NERI  OGGETTO VPN AND ANTIVIRUS   \xa0 HELLO \xa0 I HAD PROBLEM WITH VPN AND THE CITRI

In [192]:
picklefile = "filtered_df_2_email_signature.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': 'Daniel is a new employee and needs to set up his e-mail account. He has problems with VPN and the CITRIX inlogger. He leaves his computer with Team Viewer open so that if he needs something from someone, he can see it on the phone. '}], [{'summary_text': 'Beim VersuCHE EINE EMAIL AN GEORGENGLENGOICECOM ZU SCHICKENCIAO A TUTTI QUANDO LAVORO IN SMARTWORKING UTILIZZO LA VIRTUAL MACHINE E ACCEDO ALLA MAIL VIA WEB AL MOMENTO DAL WEB VEDO SOLO IL MIO INDIRIZZIO DI POSTA MENTRE DA OUTLOOK.'}], [{'summary_text': 'There are a lot of important e-mail messages that go to spam because they are sent on Monday. There are 350 e-mails in BCC. There is also a PDF READER INSTALLATO, GRAZIE PER IL TUO AIUTO and BENJAMIN. '}], [{'summary_text': 'Lucky LUCKHI wants to install Office 64 bits on MFL0210. She also needs the teamviewer ID and the license plate of her normal one.  The license plate shows 4 stars as the passwORD in the SMALL TV. '}], [{'summary_text': 'DANILE ECCO QUA, GRAZIE

# df_2_email_phishing

In [227]:
import pandas as pd
import re

In [228]:
related_words = ['Phishing', 'Pesca fraudolenta', 'Inganno online', 'Truffa informatica', 'Frode via email',
    'Phishing', 'Online scam', 'Email fraud', 'Deceptive online tactics', 'Cyber deception',
    'Phishing', 'Online-Betrug', 'Internet-Täuschung', 'E-Mail-Betrug', 'Cyber-Betrug',
    'Inganno informatico', 'Pesca di informazioni', 'Truffa digitale',
    'Digital deception', 'Information harvesting', 'Online fraud',
    'Digitale Täuschung', 'Informationsdiebstahl', 'Online-Betrug','Spam', 'Posta indesiderata', 
    'Messaggi non richiesti', 'Posta spazzatura', 'Invio massivo di email',
    'Spam', 'Unsolicited email', 'Junk mail', 'Email overload', 'Bulk messaging',
    'Spam', 'Unerwünschte E-Mails', 'Junk-Mail', 'E-Mail-Überflutung', 'Massenversand von Nachrichten',
    'Posta non desiderata', 'Messaggi di spam', 'Comunicazioni indesiderate',
    'Unwanted mail', 'Spam messages', 'Unsolicited communications',
    'Unerwünschte Post', 'Spam-Nachrichten', 'Ungebetene Kommunikation']

In [229]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_2_email_phishing = df_2_email[df_2_email['Text'].str.contains(pattern, case=False)]
filtered_df_2_email_phishing = filtered_df_2_email_phishing.reset_index(drop=True)

filtered_df_2_email_phishing

,Title,Topic,Text
0,impostazioni SPAM,"{""12"":""Standard Software: Operating systems, M...",CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM A...
1,Spam?,"{""12"":""Standard Software: Operating systems, M...",CIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA ...
2,Important mails in Outlook go all to Spam sinc...,"{""12"":""Standard Software: Operating systems, M...",HELLO MANY IMPORTANT MAILS COMING FROM SUPPLIE...
3,Controllare posta in arrivo dalla KAESER,"{""12"":""Standard Software: Operating systems, M...",OK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE HEI...
4,Spam?,"{""12"":""Standard Software: Operating systems, M...",CIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA ...


In [230]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_2_email_phishing[i:i+n] for i in range(0,len(filtered_df_2_email_phishing),n)]
    #print(list_df)

In [231]:
len(list_df)

1

In [232]:
list_df[0]

,Title,Topic,Text
0,impostazioni SPAM,"{""12"":""Standard Software: Operating systems, M...",CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM A...
1,Spam?,"{""12"":""Standard Software: Operating systems, M...",CIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA ...
2,Important mails in Outlook go all to Spam sinc...,"{""12"":""Standard Software: Operating systems, M...",HELLO MANY IMPORTANT MAILS COMING FROM SUPPLIE...
3,Controllare posta in arrivo dalla KAESER,"{""12"":""Standard Software: Operating systems, M...",OK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE HEI...
4,Spam?,"{""12"":""Standard Software: Operating systems, M...",CIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA ...


In [233]:
def put_together():
    list1=[]
    for index in range(0,1):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM ANCHE SE FACCIO NON BLOCCARE MAI DOMINIO DEL MITTENTE CHIEDO DI IMPOSTARE I SEGUENTI DOMINIO COME MAI SPAM PER LE TRE INDIRZI MAIL CHE STO USANDO INDIRIZZI MAIL CHE STO USANDO CHRISTIANSTAUDEREMICONTROLSCOM SPAREPARTSEMICONTROLSCOM SERVICEFIREEMICONTROLSCOM  DOMINI CHE NON SONO SPAM FUNGIBLESCONDALCOM NEBOLEXDE HELIOSFIRESYSTEMSUK SAMPIERANACOM IVECOGROUPCOMCIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIEHELLO MANY IMPORTANT MAILS COMING FROM SUPPLIERS AND CLIENTS ALL GO TO SPAM SINCE MONDAY REALLY INCONVENIENTOK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE  HEINZFARTHOFERKAESERCOM KOLLER FLORIAN  GERHILDTASCHKAESERCOM  LA JUNK MAIL CHE VEDIAMO NOI SULLA INBOX NON  LA STESSA DELLO SPAM CHE IN QUESTO CASO NOI UTENTI NON VEDIAMO PUOI CONFERMARE  GRAZIECIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIE



['CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM ANCHE SE FACCIO\xa0NON BLOCCARE MAI DOMINIO DEL MITTENTE CHIEDO DI IMPOSTARE I SEGUENTI DOMINIO COME MAI SPAM PER LE TRE INDIRZI MAIL CHE STO USANDO INDIRIZZI MAIL CHE STO USANDO CHRISTIANSTAUDEREMICONTROLSCOM SPAREPARTSEMICONTROLSCOM SERVICEFIREEMICONTROLSCOM  DOMINI CHE NON SONO SPAM FUNGIBLESCONDALCOM NEBOLEXDE HELIOSFIRESYSTEMSUK SAMPIERANACOM IVECOGROUPCOMCIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIEHELLO MANY IMPORTANT MAILS COMING FROM SUPPLIERS AND CLIENTS ALL GO TO SPAM SINCE MONDAY REALLY INCONVENIENTOK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE\xa0\xa0HEINZFARTHOFERKAESERCOM\xa0KOLLER FLORIAN \xa0GERHILDTASCHKAESERCOM\xa0 LA JUNK MAIL CHE VEDIAMO NOI SULLA INBOX NON  LA STESSA DELLO SPAM CHE IN QUESTO CASO NOI UTENTI NON VEDIAMO PUOI CONFERMARE  GRAZIECIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIE']

In [234]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 1):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [235]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM ANCHE SE FACCIO NON BLOCCARE MAI DOMINIO DEL MITTENTE CHIEDO DI IMPOSTARE I SEGUENTI DOMINIO COME MAI SPAM PER LE TRE INDIRZI MAIL CHE STO USANDO INDIRIZZI MAIL CHE STO USANDO CHRISTIANSTAUDEREMICONTROLSCOM SPAREPARTSEMICONTROLSCOM SERVICEFIREEMICONTROLSCOM  DOMINI CHE NON SONO SPAM FUNGIBLESCONDALCOM NEBOLEXDE HELIOSFIRESYSTEMSUK SAMPIERANACOM IVECOGROUPCOMCIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIEHELLO MANY IMPORTANT MAILS COMING FROM SUPPLIERS AND CLIENTS ALL GO TO SPAM SINCE MONDAY REALLY INCONVENIENTOK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE  HEINZFARTHOFERKAESERCOM KOLLER FLORIAN  GERHILDTASCHKAESERCOM  LA JUNK MAIL CHE VEDIAMO NOI SULLA INBOX NON  LA STESSA DELLO SPAM CHE IN QUESTO CASO NOI UTENTI NON VEDIAMO PUOI CONFERMARE  GRAZIECIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIE



In [236]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM ANCHE SE FACCIO NON BLOCCARE MAI DOMINIO DEL MITTENTE CHIEDO DI IMPOSTARE I SEGUENTI DOMINIO COME MAI SPAM PER LE TRE INDIRZI MAIL CHE STO USANDO INDIRIZZI MAIL CHE STO USANDO CHRISTIANSTAUDEREMICONTROLSCOM SPAREPARTSEMICONTROLSCOM SERVICEFIREEMICONTROLSCOM  DOMINI CHE NON SONO SPAM FUNGIBLESCONDALCOM NEBOLEXDE HELIOSFIRESYSTEMSUK SAMPIERANACOM IVECOGROUPCOMCIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIEHELLO MANY IMPORTANT MAILS COMING FROM SUPPLIERS AND CLIENTS ALL GO TO SPAM SINCE MONDAY REALLY INCONVENIENTOK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE  HEINZFARTHOFERKAESERCOM KOLLER FLORIAN  GERHILDTASCHKAESERCOM  LA JUNK MAIL CHE VEDIAMO NOI SULLA INBOX NON  LA STESSA DELLO SPAM CHE IN QUESTO CASO NOI UTENTI NON VEDIAMO PUOI CONFERMARE  GRAZIECIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIE



['CIAO TANTI DOMAIN VENGONO TRATTATI COME SPAM ANCHE SE FACCIO\xa0NON BLOCCARE MAI DOMINIO DEL MITTENTE CHIEDO DI IMPOSTARE I SEGUENTI DOMINIO COME MAI SPAM PER LE TRE INDIRZI MAIL CHE STO USANDO INDIRIZZI MAIL CHE STO USANDO CHRISTIANSTAUDEREMICONTROLSCOM SPAREPARTSEMICONTROLSCOM SERVICEFIREEMICONTROLSCOM  DOMINI CHE NON SONO SPAM FUNGIBLESCONDALCOM NEBOLEXDE HELIOSFIRESYSTEMSUK SAMPIERANACOM IVECOGROUPCOMCIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIEHELLO MANY IMPORTANT MAILS COMING FROM SUPPLIERS AND CLIENTS ALL GO TO SPAM SINCE MONDAY REALLY INCONVENIENTOK CIO SBLOCCATI TUTTI DELLA KAESER ANCHE\xa0\xa0HEINZFARTHOFERKAESERCOM\xa0KOLLER FLORIAN \xa0GERHILDTASCHKAESERCOM\xa0 LA JUNK MAIL CHE VEDIAMO NOI SULLA INBOX NON  LA STESSA DELLO SPAM CHE IN QUESTO CASO NOI UTENTI NON VEDIAMO PUOI CONFERMARE  GRAZIECIAO HO APPENA RICEVUTO QUESTA MAIL CREDO SIA SPAM CORRETTO GRAZIE']

# df_2_access

In [83]:
#df_2_access
import pandas as pd

words_to_check = ['teams', 'login', 'accesso', 'accedere', 'pw', 'password']


# Initialize an empty DataFrame to store the matching rows
df_2_access = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_access = df_2_access.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_access = df_2_access.reset_index(drop=True)

df_2_access

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\981805679.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_access = df_2_access.append(d

,Title,Topic,Text,tokens
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...,"[computer, c405, teams, persona, chiama, suona..."
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...,"[vedo, piu, link, teams, meeting, creo, appunt..."
2,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...,"[collegamento, tra, programmi, sopracitati, fi..."
3,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...,"[nostro, partner, finlandia, manu, laari, manu..."
4,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...,"[qualche, settimana, camera, computer, home, o..."
...,...,...,...,...
186,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...,"[trying, adobe, pro, will, let, had, old, inst..."
187,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...,"[accesso, vedo, 10, cartelle, drive, epowertra..."
188,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...,"[lid, rimane, password, aveva, timeout, qui, q..."
189,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...,"[update, windows, viewer, 545, 489, 361passwor..."


In [84]:
df_2_access

,Title,Topic,Text,tokens
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...,"[computer, c405, teams, persona, chiama, suona..."
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...,"[vedo, piu, link, teams, meeting, creo, appunt..."
2,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...,"[collegamento, tra, programmi, sopracitati, fi..."
3,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...,"[nostro, partner, finlandia, manu, laari, manu..."
4,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...,"[qualche, settimana, camera, computer, home, o..."
...,...,...,...,...
186,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...,"[trying, adobe, pro, will, let, had, old, inst..."
187,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...,"[accesso, vedo, 10, cartelle, drive, epowertra..."
188,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...,"[lid, rimane, password, aveva, timeout, qui, q..."
189,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...,"[update, windows, viewer, 545, 489, 361passwor..."


In [85]:
df_2_access = df_2_access.drop(columns = 'tokens')
df_2_access

,Title,Topic,Text
0,TEAMS Audio non funziona,"{""12"":""Standard Software: Operating systems, M...",COMPUTER C405 HO UN PROBLEMA CON TEAMS QUANDO...
1,Outlook Termine erstellen TEams,"{""12"":""Standard Software: Operating systems, M...",CIAO DA OGGI NON VEDO PIU IL LINK PER IL TEAMS...
2,Collegamento MS TEAMS e OUTLOOK,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO IL COLLEGAMENTO TRA I DUE PROGRAMMI...
3,MSTeams - Guest contact,"{""12"":""Standard Software: Operating systems, M...",CIAO PER FAVORE AGGIUNGERE IL NOSTRO PARTNER I...
4,Camera non funziona da casa,"{""12"":""Standard Software: Operating systems, M...",CIAO DA QUALCHE SETTIMANA HO QUESTO PROBLEMA C...
...,...,...,...
186,Downloading adobe pro,"{""12"":""Standard Software: Operating systems, M...",I AM TRYING TO INSTALL ADOBE PRO AND IT WILL N...
187,ONE DRIVE TECHNOALPIN EPOWERTRAC,"{""12"":""Standard Software: Operating systems, M...",BUONGIORNO NON HO ACCESSO NON VEDO 10 CARTEL...
188,Setup nuovo PC,"{""12"":""Standard Software: Operating systems, M...",LID RIMANE PASSWORD AVEVA TIMEOUT QUI QUELLA N...
189,Update Windows,"{""12"":""Standard Software: Operating systems, M...",HI COULD YOU HELP ME UPDATE WINDOWS TEAM VIEW...


# df_2_software

In [86]:
#df_2_software
import pandas as pd

words_to_check = ['installare', 'nav', 'citrix', 'vpn', 'edge', 'acrobat', 'prfprotokoll', 'vdi', 'server']

# Initialize an empty DataFrame to store the matching rows
df_2_software = pd.DataFrame(columns=df_2.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_2['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_2_software = df_2_software.append(df_2[word_mask])

# Reset the index of the new DataFrame
df_2_software = df_2_software.reset_index(drop=True)

df_2_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df_2_software.append(df_2[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2289260216.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_2_software = df

,Title,Topic,Text,tokens
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...,"[installare, word, excel, ed, outlook, macchin..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...,"[ciaomi, installare, pack, solid, edge, sp7gra..."
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...,"[riapro, questione, acrobat, reader, 64bit, im..."
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...,"[discusso, voce, department, installare, softw..."
...,...,...,...,...
194,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...,"[migrazione, server, sembra, andata, bene, err..."
195,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
196,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...,"[servono, servizio, trisoft, deve, mandare, em..."
197,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...,"[our, technican, unable, recive, mail, his, la..."


In [87]:
df_2_software

,Title,Topic,Text,tokens
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...,"[installare, word, excel, ed, outlook, macchin..."
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...,"[installare, dynamics, nav, pc, vpn, accedere,..."
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...,"[ciaomi, installare, pack, solid, edge, sp7gra..."
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...,"[riapro, questione, acrobat, reader, 64bit, im..."
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...,"[discusso, voce, department, installare, softw..."
...,...,...,...,...
194,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...,"[migrazione, server, sembra, andata, bene, err..."
195,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...,"[vogliamo, lndf, manda, email, caso, problemi,..."
196,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...,"[servono, servizio, trisoft, deve, mandare, em..."
197,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...,"[our, technican, unable, recive, mail, his, la..."


In [88]:
df_2_software = df_2_software.drop(columns = 'tokens')
df_2_software

,Title,Topic,Text
0,Office365 x C255,"{""12"":""Standard Software: Operating systems, M...",PREGO INSTALLARE WORD EXCEL ED OUTLOOK SU LA M...
1,VPN & Navision,"{""12"":""Standard Software: Operating systems, M...",CIAO POTETE INSTALLARE DYNAMICS NAV SUL MIO PC...
2,Please install Solid Edge SP7,"{""12"":""Standard Software: Operating systems, M...",CIAOMI POTETE PER FAVORE INSTALLARE IL NUOVO S...
3,ACROBAT READER 64-bit per deploy #140129,"{""12"":""Standard Software: Operating systems, M...",CIAO RIAPRO LA QUESTIONE ACROBAT READER 64BIT ...
4,installazione profile e creazione account,"{""12"":""Standard Software: Operating systems, M...",CIAO COME DISCUSSO A VOCE CHIEDO PER FAVORE I...
...,...,...,...
194,Aggiornamento Docfinance,"{""12"":""Standard Software: Operating systems, M...",CIAO LA MIGRAZIONE SUL NUOVO SERVER SEMBRA AND...
195,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",VOGLIAMO CHE LNDF CI MANDA UN EMAIL NEL CASO Q...
196,Indirizzo E-Mail per l'utente NDF,"{""12"":""Standard Software: Operating systems, M...",SERVONO PER UN SERVIZIO DI TRISOFT CHE DEVE MA...
197,Unable to log in on mail.,"{""12"":""Standard Software: Operating systems, M...",OUR SERVICE TECHNICAN IS UNABLE TO RECIVE MAIL...


# df_3
{"11":"Hardware, Telephony, Smartphones, Monitor, Printer,  etc"}

In [89]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')

# custom list of words to remove (greetings and articles)
custom_stopwords = ['hello', 'hi', 'a', 'an', 'the', 'is', 'di','per','grazie', 'e', 'il', 'ciao','non', 'che','la','da', 'bitte', 'mi', 'hallo', 
                    'die','come','le', 'un', 'saluti','al','technoalpin','prego', 'von', 'danke', 'please','con', 'favore', '39', 'und',
                    'fr', 'una', 'ha', 'via', 'intended', 'ich','potete','mille', 'anche', 'der', '2', 'nicht','ho', '1', 'se', '0471', 
                    'sono', 'use', 'den', 'manuel', 'del','si', 'vi','hi','id', 'ta','das', 'nuovo',
                    'ad', 'agostini','im','information','pi','0', 'buongiorno','mit','f', 'horst',
                    'action','bei','ihr', 'including', 'contains','confidential', 'individual','purpose', 
                    'protected', 'law', 'notify','taking', 'based', 'strictly','prohibited',  
                    'martin', 'thanks', 'solo','in', 'i', 'and', 'you', 'this', 'to', 'it',
                    'for', 'any', 'of', 'not','on', 'my', 'can', 'me','or', 'if', 'by','are', 'ma', 'at', 't', 
                    'have', 'should','might','am', 'delete', 'm','mio', 'alle', 'ancora', 'alla','daniele', 
                    'wwwtechnoalpincom','bolzano','from','fare','tutti','new','mir', 'hanno', 'kann','thank',
                    'matthias', '2023','zu', 'nel', 'auf', 'nella', '39100', 'bozen', 'with','hat','gi',
                    'questo', 'gli', 'sul','o', 'p', 'sie', 'teamviewer','knnt', 'lo','aggiungere', 
                    'dei', 'spa', 'help','specific', 'assegnare', 'team', 'ticket','laccesso', 
                    'serve','gre','della', 'stefan', 'service','betreff', 'n','perch', 
                    'but','suo','habe','firma','ok','no','could','oggi','su', 'ore', 'siemens',
                    'dare','pagostinistrae','200','43','posso' ,'ts','bisogno','delle', 'need', 
                    'produzione','dal','funziona','chiedo','riesco', '441', 'ist', 'neuen', 
                    'questa', 'ti','gruppo', 'that', 'regards', 'when', 'ci', 'so', 'essere', 'aprire',
                    'um','due','mai', 'usa', 'hier', 'we', 'dalle', 'de', 'en', 'zugang', 'noi', 
                    'quando', 'be','auch', 'wir', 'li' ,'cambiare', 'possibile', 'togliere',
                    'mitarbeiter','thomas', 'egregi', 'signori', 'creare', 'patrick', 'adesso',
                    'sales', 'manager','dass','gentilmente', 'mehr','gmbh', 'ept','gesendet', 'entrare', 
                    'cartella', 'problem', 'poi', 'digitale','also','du', 'problema', 'io', 
                    'es', 'here', 'pdf', 'devo', 'as','marco','41', 'provato', 'install', 
                    'tutto', 'eine','va', 'sempre', 'open','work','do','programma','one', 
                    'verificare','message','installato', '3306', 'high','ufficio', 'problemi', 
                    'uno','prone','t41', 'netprodclientsofficewlan','abbiamo', 'avere', 
                    'vulnerability', 'fa', 'francesco', 'sarebbe', 'installare', 'purtroppo', 'avete',
                    '75', 'fatto', 'multiple', 'dos', 'funktioniert','vecchio', 'domani','qualche',
                    'numero', 'piu', 'volta']

# Tokenize and remove stopwords from the 'Text' column
df_3['tokens'] = df_3['Text'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalnum() and word.lower() not in custom_stopwords])

# Flatten the tokenized words into a single list
all_words = [word for sublist in df_3['tokens'].tolist() for word in sublist]

# Count the occurrences of each word
word_counts = Counter(all_words)

# Find the most common words
most_common_words = word_counts.most_common(30)  # Change 10 to the desired number of top words
print(most_common_words)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MattiaA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('stampante', 105), ('laptop', 100), ('pc', 93), ('telefono', 48), ('docking', 44), ('stampare', 33), ('monitor', 31), ('printer', 30), ('station', 30), ('rete', 29), ('drucker', 29), ('fisso', 28), ('zebra', 25), ('cavo', 24), ('computer', 22), ('stampa', 21), ('schermo', 21), ('tastiera', 20), ('software', 20), ('pw', 20), ('possiamo', 19), ('qui', 18), ('ein', 18), ('mia', 18), ('usb', 18), ('office', 18), ('quello', 17), ('cellulare', 17), ('ne', 17), ('nuova', 17)]


# df_3_printer

In [90]:
#df_3_printer
import pandas as pd

words_to_check = ['stampante', 'stampare', 'printer', 'drucker', 'zebra', 'stampa']

# Initialize an empty DataFrame to store the matching rows
df_3_printer = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_printer = df_3_printer.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_printer = df_3_printer.reset_index(drop=True)

df_3_printer

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\245910723.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_printer = df_3_printer.

,Title,Topic,Text,tokens
0,Prego sbloccare il stampante prtserv01\F-E PCL,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO NNON RIESCO STAMPARE CAUSA DOCUMENT...,"[nnon, stampare, causa, documento, attesa, ier..."
1,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCATA,"[stampante, bloccata]"
2,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCARTA,"[stampante, bloccarta]"
3,Stampante non installato,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI STAMPARE UNA PAGINA PERO NON...,"[vorrei, stampare, pagina, pero, nessun, stamp..."
4,Stampante uffico contabilita´,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE NEL NOSTRO UFFICIO NON FUNZI...,"[stampante, nostro, venuto, fuori, manca, cart..."
...,...,...,...,...
326,Problema plotter Canon TX-3000,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI DA QUANTO STATO CAMBIATO LINDIRI...,"[quanto, stato, cambiato, lindirizzo, plotter,..."
327,Non riesco a stampare sulla nuova stampante!,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NON RIESCO A STAMPARE SULLA NUOVA STAMPANTE,"[stampare, sulla, nuova, stampante]"
328,Problemi con le stampanti ufficio EMICONTROLS,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PROBLEMI CON LE STAMPANTI UFFICIO EMICONTROLS,"[stampanti, emicontrols]"
329,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA MIA STAMPANTE NON MI PIACE,"[mia, stampante, piace]"


In [91]:
df_3_printer

,Title,Topic,Text,tokens
0,Prego sbloccare il stampante prtserv01\F-E PCL,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO NNON RIESCO STAMPARE CAUSA DOCUMENT...,"[nnon, stampare, causa, documento, attesa, ier..."
1,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCATA,"[stampante, bloccata]"
2,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCARTA,"[stampante, bloccarta]"
3,Stampante non installato,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI STAMPARE UNA PAGINA PERO NON...,"[vorrei, stampare, pagina, pero, nessun, stamp..."
4,Stampante uffico contabilita´,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE NEL NOSTRO UFFICIO NON FUNZI...,"[stampante, nostro, venuto, fuori, manca, cart..."
...,...,...,...,...
326,Problema plotter Canon TX-3000,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI DA QUANTO STATO CAMBIATO LINDIRI...,"[quanto, stato, cambiato, lindirizzo, plotter,..."
327,Non riesco a stampare sulla nuova stampante!,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NON RIESCO A STAMPARE SULLA NUOVA STAMPANTE,"[stampare, sulla, nuova, stampante]"
328,Problemi con le stampanti ufficio EMICONTROLS,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PROBLEMI CON LE STAMPANTI UFFICIO EMICONTROLS,"[stampanti, emicontrols]"
329,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA MIA STAMPANTE NON MI PIACE,"[mia, stampante, piace]"


In [92]:
df_3_printer = df_3_printer.drop(columns = 'tokens')
df_3_printer

,Title,Topic,Text
0,Prego sbloccare il stampante prtserv01\F-E PCL,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO NNON RIESCO STAMPARE CAUSA DOCUMENT...
1,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCATA
2,stampante vicino sala riunione 7,"{""11"":""Hardware, Telephony, Smartphones, Monit...",STAMPANTE BLOCCARTA
3,Stampante non installato,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI STAMPARE UNA PAGINA PERO NON...
4,Stampante uffico contabilita´,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO LA STAMPANTE NEL NOSTRO UFFICIO NON FUNZI...
...,...,...,...
326,Problema plotter Canon TX-3000,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO A TUTTI DA QUANTO STATO CAMBIATO LINDIRI...
327,Non riesco a stampare sulla nuova stampante!,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NON RIESCO A STAMPARE SULLA NUOVA STAMPANTE
328,Problemi con le stampanti ufficio EMICONTROLS,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PROBLEMI CON LE STAMPANTI UFFICIO EMICONTROLS
329,Stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",LA MIA STAMPANTE NON MI PIACE


# df_3_docking

In [93]:
#df_3_docking
import pandas as pd
words_to_check = ['docking', 'cavo', 'rete', 'lan', 'station']

# Initialize an empty DataFrame to store the matching rows
df_3_docking = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_docking = df_3_docking.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_docking = df_3_docking.reset_index(drop=True)

df_3_docking

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\770040326.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_docking = df_3_docking.

,Title,Topic,Text,tokens
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...,"[manca, adattatore, usb, lan, ne, dovrei, riti..."
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...,"[vorrei, sostituire, pc, fisso, portatile, doc..."
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...,"[larrivo, teresa, buffa, 0409, spostarmi, sar,..."
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...,"[ask, about, thing, happened, morning, went, o..."
...,...,...,...,...
164,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...,"[dati, 1726129643, x85i1w2t, cose, impostare, ..."
165,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...,"[collegamento, tra, laptopdocking, station, mo..."
166,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...,"[controllato, docking, station, dopo, laggiorn..."
167,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...,"[accendo, pc, docking, station, andare, tastie..."


In [94]:
df_3_docking

,Title,Topic,Text,tokens
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...,"[manca, adattatore, usb, lan, ne, dovrei, riti..."
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...,"[vorrei, sostituire, pc, fisso, portatile, doc..."
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...,"[larrivo, teresa, buffa, 0409, spostarmi, sar,..."
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...,"[ask, about, thing, happened, morning, went, o..."
...,...,...,...,...
164,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...,"[dati, 1726129643, x85i1w2t, cose, impostare, ..."
165,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...,"[collegamento, tra, laptopdocking, station, mo..."
166,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...,"[controllato, docking, station, dopo, laggiorn..."
167,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...,"[accendo, pc, docking, station, andare, tastie..."


In [95]:
df_3_docking = df_3_docking.drop(columns = 'tokens')
df_3_docking

,Title,Topic,Text
0,Adapter USB per LAN,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO MI MANCA ANCORA UN ADATTATORE USB PER LAN...
1,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...
2,Laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO VORREI SOSTITUIRE IL MIO PC FISSO C...
3,spostamento ufficio Michele Valbusa + inserime...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CON LARRIVO DI TERESA BUFFA IL 0409...
4,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO IM HERE TO ASK YOU ABOUT A THING THAT H...
...,...,...,...
164,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",DATI TEAMVIEWER 1726129643 X85I1W2T COSE DA I...
165,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO IL COLLEGAMENTO TRA LAPTOPDOCKING S...
166,Collegamento Docking station mDP non funziona,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO HO CONTROLLATO LA DOCKING STATION PURTROP...
167,Docking Station non funziona correttamente,"{""11"":""Hardware, Telephony, Smartphones, Monit...",QUANDO ACCENDO IL PC LA DOCKING STATION NON MI...


# df_3_tcp

In [96]:
#df_3_tcp
import pandas as pd
words_to_check = ['tcp', 'windows']

# Initialize an empty DataFrame to store the matching rows
df_3_tcp = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_tcp = df_3_tcp.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_tcp = df_3_tcp.reset_index(drop=True)

df_3_tcp

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\589532251.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_tcp = df_3_tcp.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\589532251.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_tcp = df_3_tcp.append(df_3[word_mask])


,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [97]:
df_3_tcp

,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [98]:
df_3_tcp = df_3_tcp.drop(columns= 'tokens')
df_3_tcp

,Title,Topic,Text
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...


# df_3_db 

In [99]:
#df_3_db
import pandas as pd
words_to_check = ['oracle', 'mysql', 'windows', 'mariadb']

# Initialize an empty DataFrame to store the matching rows
df_3_db = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_db = df_3_db.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_db = df_3_db.reset_index(drop=True)

df_3_db

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\2774121526.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_db = df_3_db.append(df_3[word_mask])


,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [100]:
df_3_db

,Title,Topic,Text,tokens
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...,"[servus, haben, unserem, bro, saviese, tach, e..."
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...,"[daccordo, faccio, poter, ricevere, pc, fisso,..."
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...,"[parte, windows, 11, resto, ora, appuntamento,..."
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...,"[meinem, drucker, einstellungen, festlegen, do..."
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...,"[furlotti, first, line, support, holding, 1959..."
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...,"[able, log, into, computer, username, password..."
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...,"[473713, avevamo, aggiornato, driver, sia, tra..."
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...,"[entschuldigung, aber, diese, seite, verfgbar,..."
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...,"[computer, lenovo, thinkpad, x1, carbon, 7th, ..."
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...,"[local, printer, doesnt, properly, since, upda..."


In [101]:
df_3_db = df_3_db.drop(columns = 'tokens')
df_3_db

,Title,Topic,Text
0,neuer printer einbindung nicht möglich,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SERVUS WIR HABEN HIER IN UNSEREM BRO IN SAVIES...
1,PC per ATASS-Schulung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO COME DACCORDO CON FRANCESCO FACCIO IL TIC...
2,Stampanti non funzionanti (accesso negato),"{""11"":""Hardware, Telephony, Smartphones, Monit...",A PARTE WINDOWS 11 IL RESTO FATTO ORA HO UN A...
3,Drucker Einstellung Doppelseitig und Farbe nic...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO ICH KANN BEI MEINEM DRUCKER IN DEN EINST...
4,cambio smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MANUEL FURLOTTI IT FIRST LINE SUPPORT TECHNOA...
5,ticket 381298,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO I AM NOT ABLE TO LOG INTO THIS COMPUTER...
6,Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO CON IL TICKET 473713 AVEVAMO AGGIORNATO T...
7,für die elektronische Übermittlung statistisch...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",ENTSCHULDIGUNG ABER DIESE SEITE IST NICHT VER...
8,ThinkPad Docking station no ethernet,"{""11"":""Hardware, Telephony, Smartphones, Monit...",COMPUTER LENOVO THINKPAD X1 CARBON 7TH SYSTEM ...
9,Printer out of order,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY LOCAL PRINTER DOESNT WORK PROPERLY SINCE TH...


# df_3_pc

In [102]:
#df_3_pc
import pandas as pd
words_to_check = ['laptop', 'pc', 'schermo', 'fisso', 'tastiera', 'monitor', 
                  'computer', 'cavo', 'pw', 'usb', 'telefono']


# Initialize an empty DataFrame to store the matching rows
df_3_pc = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_pc = df_3_pc.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_pc = df_3_pc.reset_index(drop=True)

df_3_pc

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\3327546749.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_pc = df_3_pc.append(df_3[word_mask])
C:\Users\Mat

,Title,Topic,Text,tokens
0,Anfrage Beschaffung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO HABE MIT NELSON BER DAS THEMA BEREITS BE...,"[nelson, ber, thema, bereits, besprochen, brau..."
1,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...,"[order, laptop, give, some, advice, which, sui..."
2,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...,"[seit, kurzem, aufgefallen, sich, mein, laptop..."
3,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
4,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...,"[valutare, sostituzione, quanto, lento, sia, a..."
...,...,...,...,...
385,sostituzione telefono con finale 457 di Harald...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PREGO SOSTITUIRE IL TELEFONO DI HARALD IN QUAN...,"[sostituire, telefono, harald, quanto, volume,..."
386,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...,"[collegarmi, telefono, fisso, telefono, 125]"
387,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI,"[telefono, fisso]"
388,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...,"[buon, pomeriggio, effetturare, chiamateverso,..."


In [103]:
df_3_pc

,Title,Topic,Text,tokens
0,Anfrage Beschaffung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO HABE MIT NELSON BER DAS THEMA BEREITS BE...,"[nelson, ber, thema, bereits, besprochen, brau..."
1,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...,"[order, laptop, give, some, advice, which, sui..."
2,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...,"[seit, kurzem, aufgefallen, sich, mein, laptop..."
3,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...,"[mattina, comunicazione, tra, docking, station..."
4,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...,"[valutare, sostituzione, quanto, lento, sia, a..."
...,...,...,...,...
385,sostituzione telefono con finale 457 di Harald...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PREGO SOSTITUIRE IL TELEFONO DI HARALD IN QUAN...,"[sostituire, telefono, harald, quanto, volume,..."
386,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...,"[collegarmi, telefono, fisso, telefono, 125]"
387,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI,"[telefono, fisso]"
388,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...,"[buon, pomeriggio, effetturare, chiamateverso,..."


In [104]:
df_3_pc = df_3_pc.drop(columns = 'tokens')
df_3_pc

,Title,Topic,Text
0,Anfrage Beschaffung,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO HABE MIT NELSON BER DAS THEMA BEREITS BE...
1,new laptop,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HI I NEED TO ORDER A NEW LAPTOP FOR ME COULD Y...
2,Probleme mit Hotspot Ausland,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO SEIT KURZEM IST MIR AUFGEFALLEN DASS SIC...
3,Docking station,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA QUESTA MATTINA NON HO COMUNICAZIONE TR...
4,laptop LENOVO X1 carbon,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DA VALUTARE LA SOSTITUZIONE IN QUANTO LEN...
...,...,...,...
385,sostituzione telefono con finale 457 di Harald...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PREGO SOSTITUIRE IL TELEFONO DI HARALD IN QUAN...
386,Collegamento telefono fisso,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTETE COLLEGARMI IL TELEFONO FISSO NUMER...
387,Telefono FISSO non funziona più,"{""11"":""Hardware, Telephony, Smartphones, Monit...",TELEFONO FISSO NON FUNZIONA PI
388,Chiamate verso l'esterno,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON POMERIGGIO NON RIESCO AD EFFETTURARE CHIA...


# df_3_vulnerabilities

In [105]:
#df_3_vulnerabilities
import pandas as pd
words_to_check = ['vulnerabilities']

# Initialize an empty DataFrame to store the matching rows
df_3_vulnerabilities = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_vulnerabilities = df_3_vulnerabilities.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_vulnerabilities = df_3_vulnerabilities.reset_index(drop=True)
df_3_vulnerabilities= df_3_vulnerabilities.drop(columns = 'tokens')
df_3_vulnerabilities

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\155574697.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_vulnerabilities = df_3_vulnerabilities.append(df_3[word_mask])


,Title,Topic,Text


In [106]:
df_3_vulnerabilities

,Title,Topic,Text


# df_3_email

In [107]:
#df_3_email
import pandas as pd
words_to_check = ['mail']

# Initialize an empty DataFrame to store the matching rows
df_3_email = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_email = df_3_email.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_email = df_3_email.reset_index(drop=True)
df_3_email= df_3_email.drop(columns = 'tokens')
df_3_email

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\750117258.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_email = df_3_email.append(df_3[word_mask])


,Title,Topic,Text
0,Numero telefonico,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON GIORNO IL MIO NUMERO TELEFONICE SEMBRA NO...
1,Nr telefono,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE POTRESTE CAMBIARE IL NUMERO DI TELEFONO ...
2,Info for ticket 582224,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO FRANCESCO THX NOW WORK FOR MY I WILL MA...
3,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I CALLED HIM AND SENT A MAIL WITH PHOTO THE SC...
4,Toner stampante ACS 1247109,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO FINITO IL TONER DELLA STAMPANTE XE...
5,Cellulare e ProCall a Laura,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO PROCEDERE CON LA RICHIESTA A LAURA ...
6,Neues ThinkPad,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO BITTE DAS THINKPAD FR BIANCA ZANONA EIN...
7,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO PER LUTILIZZO DI MAIL E TELEFONATE ...
8,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DANIELE VEDI LO SCREEN DELLA MAIL CON LAP...
9,e-mail salvata su stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTRESTE SALVARE LA MIA EMAIL NELLA STAM...


In [108]:
df_3_email

,Title,Topic,Text
0,Numero telefonico,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUON GIORNO IL MIO NUMERO TELEFONICE SEMBRA NO...
1,Nr telefono,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE POTRESTE CAMBIARE IL NUMERO DI TELEFONO ...
2,Info for ticket 582224,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO FRANCESCO THX NOW WORK FOR MY I WILL MA...
3,Screen blackout,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I CALLED HIM AND SENT A MAIL WITH PHOTO THE SC...
4,Toner stampante ACS 1247109,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO FINITO IL TONER DELLA STAMPANTE XE...
5,Cellulare e ProCall a Laura,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO PREGO PROCEDERE CON LA RICHIESTA A LAURA ...
6,Neues ThinkPad,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO BITTE DAS THINKPAD FR BIANCA ZANONA EIN...
7,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO PER LUTILIZZO DI MAIL E TELEFONATE ...
8,Smartphone,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO DANIELE VEDI LO SCREEN DELLA MAIL CON LAP...
9,e-mail salvata su stampante,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO POTRESTE SALVARE LA MIA EMAIL NELLA STAM...


# Further refinement

## look for df_3_email access

In [136]:
import pandas as pd
import re

In [137]:
related_words = ['access', 'accessing', 'accessible', 'accessed', 'accesses', 'entry', 
                 'approach', 'admission', 'entrance', 'reach', 'open', 'connect', 
                 'visit', 'retrieve', 'use','Zugang', 'Zugriff', 'zugänglich',
                 'zugriffen', 'Zugänge', 'Eingang', 'Herangehensweise', 'Zulassung',
                 'Eingang', 'erreichen', 'öffnen', 'verbinden', 'besuchen', 'abrufen', 'benutzen',
                 'accesso', 'accesso', 'accessibile', 'accesso', 'accessi', 
                 'entrata', 'approccio', 'ammissione', 'ingresso', 'raggiungere', 
                 'aprire', 'collegare', 'visitare', 'recuperare', 'usare']

In [138]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_3_email = df_3_email[df_3_email['Text'].str.contains(pattern, case=False)]
filtered_df_3_email = filtered_df_3_email.reset_index(drop=True)

filtered_df_3_email

,Title,Topic,Text
0,Mobile email access / technician Aldo Lauri,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I AM WRITING TO YOU IN THE NAME OF MY COLLEAGU...
1,mails in entrata bloccati,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICE...
2,WG: Fernzugriff Visualisierungsserver,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO WER KANN HIER DEN ZUGANG ERSTELLEN ...
3,Postage meter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO WE ARE HAVING AN ISSUE WITH THE POSTAGE ...


In [139]:
filtered_df_3_email

,Title,Topic,Text
0,Mobile email access / technician Aldo Lauri,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I AM WRITING TO YOU IN THE NAME OF MY COLLEAGU...
1,mails in entrata bloccati,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICE...
2,WG: Fernzugriff Visualisierungsserver,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO WER KANN HIER DEN ZUGANG ERSTELLEN ...
3,Postage meter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO WE ARE HAVING AN ISSUE WITH THE POSTAGE ...


In [140]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_3_email[i:i+n] for i in range(0,len(filtered_df_3_email),n)]
    #print(list_df)

In [141]:
len(list_df)

1

In [143]:
list_df[0]


,Title,Topic,Text
0,Mobile email access / technician Aldo Lauri,"{""11"":""Hardware, Telephony, Smartphones, Monit...",I AM WRITING TO YOU IN THE NAME OF MY COLLEAGU...
1,mails in entrata bloccati,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICE...
2,WG: Fernzugriff Visualisierungsserver,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HALLO WER KANN HIER DEN ZUGANG ERSTELLEN ...
3,Postage meter,"{""11"":""Hardware, Telephony, Smartphones, Monit...",HELLO WE ARE HAVING AN ISSUE WITH THE POSTAGE ...


In [144]:
def put_together():
    list1=[]
    for index in range(0,1):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
I AM WRITING TO YOU IN THE NAME OF MY COLLEAGUE ALDO LAURI WHOS CURRENTLY OUT IN A CUSTOMERS PUMPSTATION SINCE HE CHANGED HIS GENERAL LOGIN PASSWORD HE IS NOT ABLE TO HAVE PROPER EMAIL ACCESS ON HIS MOBILE NO 41 79 347 86 95 SINCE HE IS ALSO NOT ABLE TO CREATE A IT TICKET ANYMORE I AM DOING IT ON HIS BEHALF KINDLY ASKING YOU TO GET IN TOUCH WITH HIM TO SOLVE THIS PROBLEM MANY THANKS FOR YOUR HELP AND SUPPORT CHEERS PASCALMAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY HALLO   WER KANN HIER DEN ZUGANG ERSTELLEN   BESTE GRSSE MARCO   MARCO ALBIEZ SERVICE LEITER TECHNOALPIN SCHWEIZ AG  CH  6467  SCHATTDORF  ROSSGIESSENSTRASSE 1 T 41 418745 000  M 41 78 845 28 32  WWWTECHNOALPINCOM           VON DARIO CASU  GESENDET DIENSTAG 25 APRIL 2023 1516 AN MARCO ALBIEZ  BETREFF FERNZUGRIFF VISUALISIERUNGSSERVER     CAUTION THIS IS AN EXTERNAL EMAIL PLEASE TAKE CARE WHEN CLICKING LINKS OR OPE

['I AM WRITING TO YOU IN THE NAME OF MY COLLEAGUE ALDO LAURI WHOS CURRENTLY OUT IN A CUSTOMERS PUMPSTATION SINCE HE CHANGED HIS GENERAL LOGIN PASSWORD HE IS NOT ABLE TO HAVE PROPER EMAIL ACCESS ON HIS MOBILE NO 41 79 347 86 95 SINCE HE IS ALSO NOT ABLE TO CREATE A IT TICKET ANYMORE I AM DOING IT ON HIS BEHALF KINDLY ASKING YOU TO GET IN TOUCH WITH HIM TO SOLVE THIS PROBLEM MANY THANKS FOR YOUR HELP AND SUPPORT CHEERS PASCALMAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY HALLO \xa0 WER KANN HIER DEN ZUGANG ERSTELLEN \xa0 BESTE GRSSE MARCO \xa0 MARCO ALBIEZ SERVICE LEITER TECHNOALPIN SCHWEIZ AG  CH  6467  SCHATTDORF  ROSSGIESSENSTRASSE 1 T 41 418745 000  M 41 78 845 28 32  WWWTECHNOALPINCOM           VON DARIO CASU  GESENDET DIENSTAG 25 APRIL 2023 1516 AN MARCO ALBIEZ  BETREFF FERNZUGRIFF VISUALISIERUNGSSERVER   \xa0 CAUTION THIS IS AN EXTERNAL EMAIL PLEASE TAKE CARE WHEN CLICKING LINKS OR

In [150]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 1):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [151]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
I AM WRITING TO YOU IN THE NAME OF MY COLLEAGUE ALDO LAURI WHOS CURRENTLY OUT IN A CUSTOMERS PUMPSTATION SINCE HE CHANGED HIS GENERAL LOGIN PASSWORD HE IS NOT ABLE TO HAVE PROPER EMAIL ACCESS ON HIS MOBILE NO 41 79 347 86 95 SINCE HE IS ALSO NOT ABLE TO CREATE A IT TICKET ANYMORE I AM DOING IT ON HIS BEHALF KINDLY ASKING YOU TO GET IN TOUCH WITH HIM TO SOLVE THIS PROBLEM MANY THANKS FOR YOUR HELP AND SUPPORT CHEERS PASCALMAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY HALLO   WER KANN HIER DEN ZUGANG ERSTELLEN   BESTE GRSSE MARCO   MARCO ALBIEZ SERVICE LEITER TECHNOALPIN SCHWEIZ AG  CH  6467  SCHATTDORF  ROSSGIESSENSTRASSE 1 T 41 418745 000  M 41 78 845 28 32  WWWTECHNOALPINCOM           VON DARIO CASU  GESENDET DIENSTAG 25 APRIL 2023 1516 AN MARCO ALBIEZ  BETREFF FERNZUGRIFF VISUALISIERUNGSSERVER     CAUTION THIS IS AN EXTERNAL EMAIL PLEASE TAKE CARE WHEN CLICKING LINKS OR OPE

In [152]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
I AM WRITING TO YOU IN THE NAME OF MY COLLEAGUE ALDO LAURI WHOS CURRENTLY OUT IN A CUSTOMERS PUMPSTATION SINCE HE CHANGED HIS GENERAL LOGIN PASSWORD HE IS NOT ABLE TO HAVE PROPER EMAIL ACCESS ON HIS MOBILE NO 41 79 347 86 95 SINCE HE IS ALSO NOT ABLE TO CREATE A IT TICKET ANYMORE I AM DOING IT ON HIS BEHALF KINDLY ASKING YOU TO GET IN TOUCH WITH HIM TO SOLVE THIS PROBLEM MANY THANKS FOR YOUR HELP AND SUPPORT CHEERS PASCALMAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY HALLO   WER KANN HIER DEN ZUGANG ERSTELLEN   BESTE GRSSE MARCO   MARCO ALBIEZ SERVICE LEITER TECHNOALPIN SCHWEIZ AG  CH  6467  SCHATTDORF  ROSSGIESSENSTRASSE 1 T 41 418745 000  M 41 78 845 28 32  WWWTECHNOALPINCOM           VON DARIO CASU  GESENDET DIENSTAG 25 APRIL 2023 1516 AN MARCO ALBIEZ  BETREFF FERNZUGRIFF VISUALISIERUNGSSERVER     CAUTION THIS IS AN EXTERNAL EMAIL PLEASE TAKE CARE WHEN CLICKING LINKS OR OPE

['I AM WRITING TO YOU IN THE NAME OF MY COLLEAGUE ALDO LAURI WHOS CURRENTLY OUT IN A CUSTOMERS PUMPSTATION SINCE HE CHANGED HIS GENERAL LOGIN PASSWORD HE IS NOT ABLE TO HAVE PROPER EMAIL ACCESS ON HIS MOBILE NO 41 79 347 86 95 SINCE HE IS ALSO NOT ABLE TO CREATE A IT TICKET ANYMORE I AM DOING IT ON HIS BEHALF KINDLY ASKING YOU TO GET IN TOUCH WITH HIM TO SOLVE THIS PROBLEM MANY THANKS FOR YOUR HELP AND SUPPORT CHEERS PASCALMAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY HALLO \xa0 WER KANN HIER DEN ZUGANG ERSTELLEN \xa0 BESTE GRSSE MARCO \xa0 MARCO ALBIEZ SERVICE LEITER TECHNOALPIN SCHWEIZ AG  CH  6467  SCHATTDORF  ROSSGIESSENSTRASSE 1 T 41 418745 000  M 41 78 845 28 32  WWWTECHNOALPINCOM           VON DARIO CASU  GESENDET DIENSTAG 25 APRIL 2023 1516 AN MARCO ALBIEZ  BETREFF FERNZUGRIFF VISUALISIERUNGSSERVER   \xa0 CAUTION THIS IS AN EXTERNAL EMAIL PLEASE TAKE CARE WHEN CLICKING LINKS OR

In [157]:
picklefile = "filtered_df_3_email.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

[[{'summary_text': 'Aldo Luir is out in a pumpstation. He has a problem with e-mailing on his phone. There is an issue with the Postage Meter. It needs to be set up via a software called "StatIC IP". '}]]


# df_3_email_signature

In [194]:
import pandas as pd
import re

In [195]:
related_words = [
    'Firma', 'Autografo', 'Sottoscrizione', 'Segno distintivo', 'Sigla', 'Marcatura',
    'Ratifica', 'Apposizione di firma', 'Sigillo', 'Atto autografo',
    'Signature', 'Autograph', 'Mark', 'Seal', 'Sign-off',
    'Inked mark', 'Signatory', 'Handwriting', 'Personal mark',
    'Unterschrift', 'Autogramm', 'Kennzeichen', 'Siegel', 'Namenszug', 'Paraphe',
    'Besiegelung', 'Unterzeichnung', 'Handschrift', 'Eigenhändige Unterschrift'
]

In [196]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_3_email_signature = df_3_email[df_3_email['Text'].str.contains(pattern, case=False)]
filtered_df_3_email_signature = filtered_df_3_email_signature.reset_index(drop=True)

filtered_df_3_email_signature

,Title,Topic,Text
0,possible virus and email signature,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL...


In [199]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_3_email_signature[i:i+n] for i in range(0,len(filtered_df_3_email_signature),n)]
    #print(list_df)

In [200]:
len(list_df)

1

In [ ]:
list_df[0]

In [201]:
def put_together():
    list1=[]
    for index in range(0,1):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL VIRUSES ITS ACTING A BIT STRANGE   ALSO MY EMAIL SIGNATURE KEEPS DISAPPEARING IT WILL ONLY BE ACTIVE IF I AM CONNECTED TO THE VPN AND I UPDATE IT WITH THE SYMPREX EMAIL MANAGER



['WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL VIRUSES ITS ACTING A BIT STRANGE\xa0  ALSO MY EMAIL SIGNATURE KEEPS DISAPPEARING IT WILL ONLY BE ACTIVE IF I AM CONNECTED TO THE VPN AND I UPDATE IT WITH THE SYMPREX EMAIL MANAGER']

In [202]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 1):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [203]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL VIRUSES ITS ACTING A BIT STRANGE   ALSO MY EMAIL SIGNATURE KEEPS DISAPPEARING IT WILL ONLY BE ACTIVE IF I AM CONNECTED TO THE VPN AND I UPDATE IT WITH THE SYMPREX EMAIL MANAGER



In [204]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL VIRUSES ITS ACTING A BIT STRANGE   ALSO MY EMAIL SIGNATURE KEEPS DISAPPEARING IT WILL ONLY BE ACTIVE IF I AM CONNECTED TO THE VPN AND I UPDATE IT WITH THE SYMPREX EMAIL MANAGER



['WOULD LIKE TO HAVE MY PC SCANNED FOR POTENTIAL VIRUSES ITS ACTING A BIT STRANGE\xa0  ALSO MY EMAIL SIGNATURE KEEPS DISAPPEARING IT WILL ONLY BE ACTIVE IF I AM CONNECTED TO THE VPN AND I UPDATE IT WITH THE SYMPREX EMAIL MANAGER']

# df_3_email_phishing

In [243]:
import pandas as pd
import re

In [244]:
related_words = ['Phishing', 'Pesca fraudolenta', 'Inganno online', 'Truffa informatica', 'Frode via email',
    'Phishing', 'Online scam', 'Email fraud', 'Deceptive online tactics', 'Cyber deception',
    'Phishing', 'Online-Betrug', 'Internet-Täuschung', 'E-Mail-Betrug', 'Cyber-Betrug',
    'Inganno informatico', 'Pesca di informazioni', 'Truffa digitale',
    'Digital deception', 'Information harvesting', 'Online fraud',
    'Digitale Täuschung', 'Informationsdiebstahl', 'Online-Betrug','Spam', 'Posta indesiderata', 
    'Messaggi non richiesti', 'Posta spazzatura', 'Invio massivo di email',
    'Spam', 'Unsolicited email', 'Junk mail', 'Email overload', 'Bulk messaging',
    'Spam', 'Unerwünschte E-Mails', 'Junk-Mail', 'E-Mail-Überflutung', 'Massenversand von Nachrichten',
    'Posta non desiderata', 'Messaggi di spam', 'Comunicazioni indesiderate',
    'Unwanted mail', 'Spam messages', 'Unsolicited communications',
    'Unerwünschte Post', 'Spam-Nachrichten', 'Ungebetene Kommunikation']

In [245]:
pattern = r'\b(?:' + '|'.join(related_words) + r')\b'

# Filter the DataFrame to keep only rows containing the related words
filtered_df_3_email_phishing = df_3_email[df_3_email['Text'].str.contains(pattern, case=False)]
filtered_df_3_email_phishing = filtered_df_3_email_phishing.reset_index(drop=True)

filtered_df_3_email_phishing

,Title,Topic,Text
0,mails in entrata bloccati,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICE...


In [246]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [filtered_df_3_email_phishing[i:i+n] for i in range(0,len(filtered_df_3_email_phishing),n)]

In [247]:
len(list_df)

1

In [248]:
list_df[0]

,Title,Topic,Text
0,mails in entrata bloccati,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICE...


In [249]:
def put_together():
    list1=[]
    for index in range(0,1):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


summ_block = put_together()
summ_block

Result 0:
MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY



['MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY']

In [250]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 1):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [251]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY



In [252]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY



['MAILS IN ENTRATA VENGONO BLOCCATI NON GLI RICEVO NEANCHE NEL SPAM PREGO VERIFICARE  DPAPADOPOULOSGPAPADOPOULOSCOMCY GPAPADOPOULOSGPAPADOPOULOSCOMCY']

# df_3_software

In [109]:
#df_3_software
import pandas as pd
words_to_check = ['software', 'server']

# Initialize an empty DataFrame to store the matching rows
df_3_software = pd.DataFrame(columns=df_3.columns)

# Iterate through the words to check
for word in words_to_check:
    # Create a boolean mask for the current word
    word_mask = df_3['Text'].str.contains(word, case=False)
    
    # Append rows containing the current word to the new DataFrame
    df_3_software = df_3_software.append(df_3[word_mask])

# Reset the index of the new DataFrame
df_3_software = df_3_software.reset_index(drop=True)
df_3_software= df_3_software.drop(columns = 'tokens')
df_3_software

C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\916185259.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_software = df_3_software.append(df_3[word_mask])
C:\Users\MattiaA\AppData\Local\Temp\ipykernel_14388\916185259.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_3_software = df_3_software.append(df_3[word_mask])


,Title,Topic,Text
0,PC running slow.,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY PC HAS BEEN RUNNING VERY SLOW FOR THE PAST ...
1,Monitor non si collega con il portatile,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO STAMATTINA NON RIESCO A COLLEGARMI COL MO...
2,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE SE POSSIBILE CHIEDEREI CHE FRANCESCO O M...
3,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PER ME VA BENE PER DOMANI VIENE CONCLUSO LA CO...
4,Neues Notebook SPS-Programmierer TAEE Peter Št...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WIE VON HANNES SIMONINI ANGEFORDERT BESTELLE...
5,Sostituzione PC,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CHIEDO GENTILMENTE LA SOSTITUZIONE ...
6,PC C 238 startet nicht mehr,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NAVISION TEAMVIEWER DIE WEETECH SOFTWARE MSSEN...
7,PC Showroom,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAOPOTRESTE PER FAVORE METTERE A DISPOSIZIONE...
8,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...
9,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...


In [110]:
df_3_software

,Title,Topic,Text
0,PC running slow.,"{""11"":""Hardware, Telephony, Smartphones, Monit...",MY PC HAS BEEN RUNNING VERY SLOW FOR THE PAST ...
1,Monitor non si collega con il portatile,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAO STAMATTINA NON RIESCO A COLLEGARMI COL MO...
2,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",SALVE SE POSSIBILE CHIEDEREI CHE FRANCESCO O M...
3,Modula Scaffale,"{""11"":""Hardware, Telephony, Smartphones, Monit...",PER ME VA BENE PER DOMANI VIENE CONCLUSO LA CO...
4,Neues Notebook SPS-Programmierer TAEE Peter Št...,"{""11"":""Hardware, Telephony, Smartphones, Monit...",WIE VON HANNES SIMONINI ANGEFORDERT BESTELLE...
5,Sostituzione PC,"{""11"":""Hardware, Telephony, Smartphones, Monit...",BUONGIORNO CHIEDO GENTILMENTE LA SOSTITUZIONE ...
6,PC C 238 startet nicht mehr,"{""11"":""Hardware, Telephony, Smartphones, Monit...",NAVISION TEAMVIEWER DIE WEETECH SOFTWARE MSSEN...
7,PC Showroom,"{""11"":""Hardware, Telephony, Smartphones, Monit...",CIAOPOTRESTE PER FAVORE METTERE A DISPOSIZIONE...
8,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...
9,Scanner defekt,"{""11"":""Hardware, Telephony, Smartphones, Monit...",GLI SCANNER CON LA STAZIONE DI FRANZ SONO TUTT...


# SUMMARIZATION

# Creation of blocks 

## df_1_email

In [111]:
pip install --upgrade requests


Note: you may need to restart the kernel to use updated packages.


In [112]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.


In [113]:
#specify number of rows in each chunk
n=2
#split DataFrame into chunks
list_df = [df_1_email[i:i+n] for i in range(0,len(df_1_email),n)]
    #print(list_df)

In [114]:
len(list_df)

276

In [115]:
list_df[274]

,Title,Topic,Text
548,Licenze CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTREMMO PER FAVORE CAMBIARE LA LICENZA DI...
549,Licenza CRM,"{""16"":""User identity management, Permissions, ...",CIAOPOTETE PER FAVORE ASSEGNARE A PATRICK SCHG...


In [116]:
def put_together():
    list1=[]
    for index in range(0,274):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [117]:
summ_block = put_together()
summ_block

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

['I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS\xa0 I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED\xa0 ERROR CODE 80090016HI KARIMA GUERBAA\xa0IS ABSENT UNTIL SEPTEMBER 15\xa0SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS',
 'HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT\xa0 THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN\xa0SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE ',
 'HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE I

In [118]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 274):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [119]:
blocks_to_skip = [4,5,47,110,140,168,169,173,174,176,195,204,234,241,242,244,248,250]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

In [120]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS  I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED  ERROR CODE 80090016HI KARIMA GUERBAA IS ABSENT UNTIL SEPTEMBER 15 SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS

Result 1:
HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT  THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE 

Result 2:
HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THE

['I AM RECEIVING AN ERROR MESSAGE ON SURFACE WHEN OPENING OUTLOOK TEAMS OTHER COMPUTER PROGRAMS\xa0 I AM NOT RECEIVING EMAILS OR CAN SEND ANY EMAILS NO SURFACE PROGRAMS ARE WORKING CANNOT COPY ANYTHING THE MESSAGE ON SURFACE IS YOUR COMPUTER S TRUSTED PLATFORM MODULE HAS MALFUNCTIONED\xa0 ERROR CODE 80090016HI KARIMA GUERBAA\xa0IS ABSENT UNTIL SEPTEMBER 15\xa0SICK LEAVE COULD YOU WRITE A MESSAGE OF ABSENCE ON HIS OUTLOOK EMAIL IN FRENCH BONJOUR JE SUIS ACTUELLEMENT ABSENTE ET SERAI DE RETOUR LE 18 SEPTEMBRE EN CAS DURGENCE VOUS POUVEZ CONTACTER  COMPTAFOURNISSEURTECHNOALPINCOM BIEN CORDIALEMENT BEST REGARDS',
 'HELLO I WOULD LIKE TO RETRIEVE MY OUTLOOK 2022 MESSAGES THANK YOUHI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE IS EVEN A NEW FAULT\xa0 THE MAILBOX IS SHOWING ME MORE UNREAD MESSAGES THAN I CAN\xa0SEE IN THE INBOX  COULD YOU PLEASE CHECK THANK YOU IN ADVANCE  DRTHE ',
 'HI I AM VERY SORRY BUT OUTLOOK IS STILL CLOSING BY ITS OWN  SINCE TODAY THERE I

In [121]:
!pip install torch
!pip install transformers

In [122]:
import pickle

In [123]:
picklefile = "df_1_email_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block_test = pickle.load(pkl_rb_obj)
print(summ_block_test)

[[{'summary_text': "There is an error message on Surface when opening Outlook Team. Karima Guerba is absent until September 15th. He will send his e-mail in French to his contact in France in order to put a message of absence on his Outlook team's website."}], [{'summary_text': 'The Outlook 2022 is still not closing. There is a new problem with the mailbox showing more messages than in the inbox. The person responsible will check the inbox in order to send it to the Outlook 2022 email in advance. It is not working properly yet. '}], [{'summary_text': "The Outlook is still not closing. There is a new problem with the e-mail. Daniela's employment ends tomorrow, but her company email will still receive the messages we need for accountancy.   It's not clear how it works in other departments. "}], [{'summary_text': "Andres Schwingershack sent a mesage to the office of Lorsquoffery and received a late reply from Manuel. He is a member of the Board of Directors of the Lawyer's Association of 

# df_1_rights

In [124]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_rights[i:i+n] for i in range(0,len(df_1_rights),n)]
    #print(list_df)

In [125]:
len(list_df)

64

In [126]:
list_df[62]

,Title,Topic,Text
372,Accesso G:\Neubau Gebäude,"{""16"":""User identity management, Permissions, ...",BUONGIORNO PER CORTESIA POTETE DARE ALLA ANNI ...
373,CRM accesso bloccato,"{""16"":""User identity management, Permissions, ...",ACCESSO BLOCCATO HO USATO SCHGAGULERPATRICKTE...
374,Acesso Magazzino via Siemens,"{""16"":""User identity management, Permissions, ...",CIAOMI POTETE DARE LACCESSO CON IL MIO BADGE A...
375,WG: test,"{""16"":""User identity management, Permissions, ...",QUELLA DELLA HOLDING NON FUNZIONA ANCORA ...
376,Zugang auf Postfach AU,"{""16"":""User identity management, Permissions, ...",CIAO JULIAN ZOZIN HA ACCESSO A SERVICETECHNOAL...
377,accesso ai due testbench in produzione,"{""16"":""User identity management, Permissions, ...",IO E DANIELE F NON RIUSCIAMO PI AD ACCEDERE AI...


In [127]:
def put_together():
    list1=[]
    for index in range(0,62):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [128]:
summ_block = put_together()
summ_block

Result 0:
BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIANA AMODIO DOVREBBE ESSERE NR 10147 IL DIRITTO DI APRIRE LA PORTA INTERNA CHE CONNETTE LUFFICIO CON LA PRODUZIONE DOVE C LA MACCHINA DEL CAFF NEGLI STESSI ORARI COME PU APRIRE LE ALTRE PORTECIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA CON GLI ACCESSI 847  RESERVE 1  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 842  RESERVE 2  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 868  RESERVE 3  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 127  RESERVE 4  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 401  RESERVE 5  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 611  RESERVE 6  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 10169  RESERVE 7  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 893  RESERVE 8  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 1062  RESERVE 9  ACCESSO STANDARD HQ  PRODUZIONE 1072  RESERVE 10  ACCESSO STANDARD HQ  PRODUZIONE  GRAZIE HANNES HALLOBITTE FR UNSEREN NEU

['BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIANA AMODIO DOVREBBE ESSERE NR 10147 IL DIRITTO DI APRIRE LA PORTA INTERNA CHE CONNETTE LUFFICIO CON LA PRODUZIONE DOVE C LA MACCHINA DEL CAFF NEGLI STESSI ORARI COME PU APRIRE LE ALTRE PORTECIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA\xa0CON GLI ACCESSI 847  RESERVE 1  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 842  RESERVE 2 \xa0ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 868  RESERVE 3\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 127  RESERVE 4\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 401  RESERVE 5\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 611  RESERVE 6\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 10169  RESERVE 7\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 893  RESERVE 8\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 1062  RESERVE 9\xa0 ACCESSO STANDARD HQ  PRODUZIONE 1072  RESERVE 10\xa0 ACCESSO STANDARD HQ  PRODUZIONE  GRAZIE HANNES HAL

In [129]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 62):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [130]:
blocks_to_skip = [12,16,39,43,51,58,61]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

Result 0:
BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIANA AMODIO DOVREBBE ESSERE NR 10147 IL DIRITTO DI APRIRE LA PORTA INTERNA CHE CONNETTE LUFFICIO CON LA PRODUZIONE DOVE C LA MACCHINA DEL CAFF NEGLI STESSI ORARI COME PU APRIRE LE ALTRE PORTECIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA CON GLI ACCESSI 847  RESERVE 1  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 842  RESERVE 2  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 868  RESERVE 3  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 127  RESERVE 4  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 401  RESERVE 5  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 611  RESERVE 6  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 10169  RESERVE 7  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 893  RESERVE 8  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 1062  RESERVE 9  ACCESSO STANDARD HQ  PRODUZIONE 1072  RESERVE 10  ACCESSO STANDARD HQ  PRODUZIONE  GRAZIE HANNES HALLOBITTE FR UNSEREN NEU

In [131]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

Result 0:
BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIANA AMODIO DOVREBBE ESSERE NR 10147 IL DIRITTO DI APRIRE LA PORTA INTERNA CHE CONNETTE LUFFICIO CON LA PRODUZIONE DOVE C LA MACCHINA DEL CAFF NEGLI STESSI ORARI COME PU APRIRE LE ALTRE PORTECIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA CON GLI ACCESSI 847  RESERVE 1  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 842  RESERVE 2  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 868  RESERVE 3  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 127  RESERVE 4  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 401  RESERVE 5  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 611  RESERVE 6  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 10169  RESERVE 7  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 893  RESERVE 8  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 1062  RESERVE 9  ACCESSO STANDARD HQ  PRODUZIONE 1072  RESERVE 10  ACCESSO STANDARD HQ  PRODUZIONE  GRAZIE HANNES HALLOBITTE FR UNSEREN NEU

['BUONGIORNO VI PREGO DI DARE AL BADGE DI ADRIANA AMODIO DOVREBBE ESSERE NR 10147 IL DIRITTO DI APRIRE LA PORTA INTERNA CHE CONNETTE LUFFICIO CON LA PRODUZIONE DOVE C LA MACCHINA DEL CAFF NEGLI STESSI ORARI COME PU APRIRE LE ALTRE PORTECIAO PREGO RINOMINARE I SEGUENTI BADGE IN KABA\xa0CON GLI ACCESSI 847  RESERVE 1  ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 842  RESERVE 2 \xa0ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 868  RESERVE 3\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 127  RESERVE 4\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 401  RESERVE 5\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 611  RESERVE 6\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 10169  RESERVE 7\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 893  RESERVE 8\xa0 ACCESSO STANDARD HQ SOLO ENTRATA GENERALE  ASCENSORI 1062  RESERVE 9\xa0 ACCESSO STANDARD HQ  PRODUZIONE 1072  RESERVE 10\xa0 ACCESSO STANDARD HQ  PRODUZIONE  GRAZIE HANNES HAL

In [132]:
picklefile = "df_1_rights_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block_test = pickle.load(pkl_rb_obj)
print(summ_block_test)

[[{'summary_text': 'E.G. SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10147 is a GRAZIE SALUTI HORSTCIAO and IL MIO BADGE is a TUTTI GLI ACCESSI AUTORIZZAZIONI. '}], [{'summary_text': 'GRAZIE SALUTI HORSTEGREGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10004 PERCH MESSNER LUKAS AVEVA IERI IL SUO ULTIMO GIORNO DI LAVORO GRAZie SALUTi HORTPREGO FARE NUOVO PER MICHELE MARSONET.'}], [{'summary_text': 'EGI SIGNORI VI PREGO DI DISATTIVARE IL BADGE N 10150 PERCH ALEX GASSER TERMINA OGGI IL SUO TIROCINO IN ANTICIPO GRAZIE SALUTI HORSTCIAO RAGAZZI. '}], [{'summary_text': 'Clemens Pintt, ACCESSO GARAGE, ABILITARE BADGECIAO VI PREGO DI CREARE, UN BADGE PER JONSTON DARREN IN CASA FINO 010923 MENSA PAGATA PROFILO, IL BADGE N 10050 PERCH OGGI TERMINA, IL TIROCINIO DI GAMPER SIMON GRAZIE SALUTI HORST PREGO DISATTIVARE, IL Badge ADESSO PERCH HA LAVORATO SOLO FINO ALLE 1200, IL GOODGE A CASA POTETE PREPARARE UNO'}], [{'summary_text': 'CIAO PREGO VI CHIEDERE DI PREPARARE UN BADGE PER FELIX DILGER PER 170

# df_1_computer


In [133]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_computer[i:i+n] for i in range(0,len(df_1_computer),n)]
    #print(list_df)

In [134]:
len(list_df)

22

In [135]:
list_df[22]

IndexError: list index out of range

In [ ]:
def put_together():
    list1=[]
    for index in range(0,22):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 22):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = [2,4,12]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_1_computer_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_1_software

In [ ]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_1_software[i:i+n] for i in range(0,len(df_1_software),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[28]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,28):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 28):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = [3,4,12]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_1_software_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_2_pc

In [ ]:
#specify number of rows in each chunk
n=7
#split DataFrame into chunks
list_df = [df_2_pc[i:i+n] for i in range(0,len(df_2_pc),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[39]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,39):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 39):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
blocks_to_skip = [16]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
import pickle

In [ ]:
picklefile = "df_2_pc_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_2_email

In [ ]:
#specify number of rows in each chunk
n=5
#split DataFrame into chunks
list_df = [df_2_email[i:i+n] for i in range(0,len(df_2_email),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[93]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,93):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 93):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = [0,1,3,4,12,14,22,25,27,29,47,62,63,66,67,69,70,77]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_2_email_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_2_access

In [ ]:
#specify number of rows in each chunk
n=7
#split DataFrame into chunks
list_df = [df_2_access[i:i+n] for i in range(0,len(df_2_access),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[27]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,27):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 27):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = [17]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_2_access_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_2_software 

In [ ]:
#specify number of rows in each chunk
n=7
#split DataFrame into chunks
list_df = [df_2_software[i:i+n] for i in range(0,len(df_2_software),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[28]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,29):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 29):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
blocks_to_skip = [15,27,28]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)


In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_2_software_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3

# df_3_printer 

In [ ]:
#specify number of rows in each chunk
n=9
#split DataFrame into chunks
list_df = [df_3_printer[i:i+n] for i in range(0,len(df_3_printer),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[36]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,36):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block


In [ ]:
picklefile = "df_3_printer_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3_docking

In [ ]:
#specify number of rows in each chunk
n=4
#split DataFrame into chunks
list_df = [df_3_docking[i:i+n] for i in range(0,len(df_3_docking),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[41]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,41):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 41):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = [27,28]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_3_docking_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3_tcp

In [ ]:
#specify number of rows in each chunk
n=3
#split DataFrame into chunks
list_df = [df_3_tcp[i:i+n] for i in range(0,len(df_3_tcp),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[3]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,4):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 4):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
blocks_to_skip = [1]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_3_tcp_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3_db

In [ ]:
#specify number of rows in each chunk
n=3
#split DataFrame into chunks
list_df = [df_3_db[i:i+n] for i in range(0,len(df_3_db),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[3]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,4):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 4):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
blocks_to_skip = [1]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_3_db_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3_pc

In [ ]:
#specify number of rows in each chunk
n=4
#split DataFrame into chunks
list_df = [df_3_pc[i:i+n] for i in range(0,len(df_3_pc),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[96]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,96):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block

In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 96):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

In [ ]:
blocks_to_skip = []  # List of block numbers to skip
result_list = put_together(blocks_to_skip)


In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test


In [ ]:
picklefile = "df_3_pc_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

# df_3_vulnerabilities

In [ ]:
#empty

# df_3_email

In [ ]:
#specify number of rows in each chunk
n=6
#split DataFrame into chunks
list_df = [df_3_email[i:i+n] for i in range(0,len(df_3_email),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[5]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,5):
        res = ''.join(map(str,list_df[index]['Text']))
        #print('Result:\n',res,'\n')
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1

summ_block = put_together()
summ_block


In [ ]:
## Skip the block if it's in the list of blocks to skip
def put_together(blocks_to_skip=[]):
    list1 = []
    for index in range(0, 5):
        if index in blocks_to_skip:
            continue  # Skip the block if it's in the list of blocks to skip

        res = ''.join(map(str, list_df[index]['Text']))
        print(f'Result {index}:\n{res}\n')
        list1.append(res)
    return list1


In [ ]:
blocks_to_skip = [2]  # List of block numbers to skip
result_list = put_together(blocks_to_skip)

In [ ]:
summ_block_test = put_together(blocks_to_skip)
summ_block_test

In [ ]:
picklefile = "df_3_email_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)

### df_3_software

In [ ]:
#specify number of rows in each chunk
n=3
#split DataFrame into chunks
list_df = [df_3_software[i:i+n] for i in range(0,len(df_3_software),n)]
    #print(list_df)

In [ ]:
len(list_df)

In [ ]:
list_df[11]

In [ ]:
def put_together():
    list1=[]
    for index in range(0,12):
        res = ''.join(map(str,list_df[index]['Text']))
        print('Result:\n',res,'\n')
        list1.append(res)
    return list1

In [ ]:
summ_block = put_together()
summ_block

In [ ]:
import pickle

In [ ]:
picklefile = "df_3_software_summ.pkl"
with open(picklefile, "rb") as pkl_rb_obj:
	summ_block = pickle.load(pkl_rb_obj)
print(summ_block)